In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog  # Import simpledialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os

# --- Configuration Constants ---

# How many pixels of movement is "not still"?
# This will need tuning based on your camera resolution and distance.
STILLNESS_THRESHOLD_PX = 10

# How many consecutive "still" frames trigger an alarm?
# (e.g., 100 frames at ~20fps is ~5 seconds)
SLEEP_ALARM_FRAMES = 100

# New constant for tracking
# If the closest-found face is further than this from the last frame,
# we assume it's a new person or the tracker lost the original.
MAX_TRACKING_JUMP_PX = 150

class SleepingAlertApp:
    def __init__(self, window):
        """
        Initialize the application.
        """
        self.window = window
        self.window.title("Sleeping Alert System (Dev Phase)")
        self.window.geometry("800x700")

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- Face Detection ---
        # Use the built-in Haar Cascade data from OpenCV
        try:
            face_cascade_path = os.path.join(cv2.data.haarcascades, 'haarcascade_frontalface_default.xml')
            if not os.path.exists(face_cascade_path):
                raise IOError("Haar cascade file not found.")
            self.face_cascade = cv2.CascadeClassifier(face_cascade_path)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load Haar Cascade: {e}\nPlease ensure OpenCV is correctly installed.")
            self.window.destroy()
            return

        # --- "Sleeping" Logic State ---
        self.last_face_center = None
        self.stillness_counter = 0
        
        # --- New Registration State ---
        self.is_monitoring = False  # Are we actively monitoring a registered person?
        self.registered_face_center_guess = None # The last known center of the monitored person
        self.registered_person_name = None # Store the registered person's name
        self.current_frame_for_registration = None # Temp stores frame for registration
        self.registration_lock = threading.Lock() # Lock for accessing the frame

        # --- GUI Elements ---
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Video Display Label
        self.video_label = tk.Label(self.main_frame, bg="black")
        self.video_label.pack(fill=tk.BOTH, expand=True)

        # Status Label
        self.status_text = tk.StringVar()
        self.status_text.set("Ready. Press 'Start Camera' to begin.")
        self.status_label = tk.Label(self.main_frame, textvariable=self.status_text, font=("Arial", 14), pady=10)
        self.status_label.pack()

        # Control Buttons
        self.button_frame = tk.Frame(self.main_frame)
        self.button_frame.pack()

        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, font=("Arial", 12), width=15)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, font=("Arial", 12), width=15)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        # --- New Registration Buttons ---
        self.register_button = tk.Button(self.button_frame, text="Register & Monitor", command=self.register_person, font=("Arial", 12), width=18)
        self.register_button.pack(side=tk.LEFT, padx=5)

        self.clear_button = tk.Button(self.button_frame, text="Clear Registration", command=self.clear_registration, font=("Arial", 12), width=18)
        self.clear_button.pack(side=tk.LEFT, padx=5)


        # Set up the close protocol
        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        """
        Starts the video capture in a new thread.
        """
        if self.is_running:
            return

        try:
            self.cap = cv2.VideoCapture(0)  # Use 0 for laptop webcam
            if not self.cap.isOpened():
                raise IOError("Cannot open webcam.")
            
            self.is_running = True
            
            # Start the video processing thread
            # daemon=True ensures the thread will close when the main app closes
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            
            # self.start_button.config(state=tk.DISABLED)
            # self.stop_button.config(state=tk.NORMAL)
            # self.register_button.config(state=tk.NORMAL) # Enable registration
            self.status_text.set("Camera running. Click 'Register & Monitor'.")
        
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            if self.cap:
                self.cap.release()

    def stop_video_stream(self):
        """
        Signals the video loop to stop.
        """
        # Add guard clause
        if not self.is_running:
            return
            
        self.is_running = False
        
        # The thread will see self.is_running is False and exit
        # We wait a moment for the thread to finish
        if self.video_thread:
            self.video_thread.join(timeout=0.5) 
            
        if self.cap:
            self.cap.release()

        # self.start_button.config(state=tk.NORMAL)
        # self.stop_button.config(state=tk.DISABLED)
        self.status_text.set("Camera stopped.")
        self.video_label.config(image=None) # Clear the image
        
        # --- Reset all states ---
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None # Clear name
        self.last_face_center = None
        self.stillness_counter = 0
        # self.register_button.config(state=tk.DISABLED)
        # self.clear_button.config(state=tk.DISABLED)

    def video_loop(self):
        """
        The main loop for video processing. Runs in a separate thread.
        """
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    self.status_text.set("Error: Can't read from camera.")
                    time.sleep(0.5)
                    continue

                # Flip for a "mirror" view, which is more intuitive
                frame = cv2.flip(frame, 1)
                
                # Store a copy of the frame for the registration function
                with self.registration_lock:
                    self.current_frame_for_registration = frame.copy()
                
                # Process the frame
                processed_frame, status = self.process_frame_logic(frame)

                # Update the status text
                self.status_text.set(status)

                # Convert the OpenCV (BGR) frame to a PIL (RGB) image
                cv_img = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                
                # Resize image to fit the label (optional, but good for layout)
                w, h = self.video_label.winfo_width(), self.video_label.winfo_height()
                if w > 1 and h > 1: # Avoid division by zero on init
                     pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)

                # Convert PIL image to Tkinter-compatible image
                imgtk = ImageTk.PhotoImage(image=pil_img)

                # Update the video label in the GUI
                # This must be done from the main thread, but tkinter seems to handle this call
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

            except Exception as e:
                print(f"Error in video loop: {e}")
                self.is_running = False
            
            # Control loop speed slightly
            time.sleep(0.01) # ~100fps theoretical max, but processing will slow it down

        print("Video loop stopped.")

    def register_person(self):
        """
        Registers the largest face currently in the frame for monitoring.
        """
        # --- Add Guard Clauses ---
        if not self.is_running:
            messagebox.showwarning("Error", "Camera is not running. Please start the camera first.")
            return
        
        if self.is_monitoring:
            messagebox.showwarning("Error", "A person is already being monitored. Please clear the registration first.")
            return

        with self.registration_lock:
            if self.current_frame_for_registration is None:
                messagebox.showwarning("Registration Error", "Camera not ready. Please try again.")
                return
            
            # Use the stored frame to find a face
            gray = cv2.cvtColor(self.current_frame_for_registration, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(50, 50)
            )

        if len(faces) == 0:
            messagebox.showwarning("Registration Error", "No person detected in the frame. Please face the camera and try again.")
            return

        # Register the largest face
        # We sort by area to be sure we get the main person
        faces_by_area = sorted(faces, key=lambda f: f[2] * f[3], reverse=True)
        (x, y, w, h) = faces_by_area[0]

        center_x = x + w // 2
        center_y = y + h // 2
        
        # --- NEW: Ask for the person's name ---
        name = simpledialog.askstring("Register Person", "Enter the person's name:", parent=self.window)
        
        if not name:
            messagebox.showwarning("Registration Cancelled", "Registration was cancelled (no name provided).")
            return
        
        # --- Lock in the registration ---
        self.is_monitoring = True
        self.registered_person_name = name
        self.registered_face_center_guess = (center_x, center_y)
        self.last_face_center = (center_x, center_y) # Start stillness check from this point
        self.stillness_counter = 0

        # Update GUI
        # self.register_button.config(state=tk.DISABLED)
        # self.clear_button.config(state=tk.NORMAL)
        self.status_text.set(f"Person registered: {self.registered_person_name}. Actively monitoring.")
        messagebox.showinfo("Registration Complete", f"{self.registered_person_name} registered. Monitoring has started.")

    def clear_registration(self):
        """
        Clears the current registration and stops monitoring.
        """
        # --- Add Guard Clause ---
        if not self.is_monitoring:
            messagebox.showwarning("Info", "No person is currently registered.")
            return
            
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None # Clear name
        self.last_face_center = None
        self.stillness_counter = 0

        # Update GUI
        # if self.is_running: # Only enable if camera is on
        #     self.register_button.config(state=tk.NORMAL)
        # self.clear_button.config(state=tk.DISABLED)
        self.status_text.set("Monitoring stopped. Ready to register a new person.")


    def process_frame_logic(self, frame):
        """
        Detects faces and applies monitoring logic based on registration status.
        This function replaces the old 'detect_sleeping'.
        """
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(50, 50) # Don't detect tiny faces
        )

        # If not monitoring, just show all faces found
        if not self.is_monitoring:
            current_status = "Ready to register."
            if self.is_running: # Check if camera is on
                current_status = "Click 'Register & Monitor' to begin."
            
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2) # Light blue box
            
            return frame, current_status

        # --- If we ARE monitoring ---
        
        if len(faces) == 0:
            # We were monitoring, but now the person is gone
            current_status = f"MONITORING: {self.registered_person_name} lost!"
            self.last_face_center = None # Stop stillness counter
            self.stillness_counter = 0
            return frame, current_status

        # Find the face closest to our last known center
        min_dist = float('inf')
        tracked_face = None
        current_center = None

        for (x, y, w, h) in faces:
            center = (x + w // 2, y + h // 2)
            dist = np.linalg.norm(np.array(center) - np.array(self.registered_face_center_guess))
            
            if dist < min_dist:
                min_dist = dist
                tracked_face = (x, y, w, h)
                current_center = center
        
        # Check if the closest face is "reasonably" close.
        # If not, it's probably a different person.
        if min_dist > MAX_TRACKING_JUMP_PX:
             # The person we found is too far from the last spot.
             current_status = f"MONITORING: {self.registered_person_name} lost! (New face detected)"
             self.last_face_center = None
             self.stillness_counter = 0
             # Draw a box on the *new* face
             (x, y, w, h) = tracked_face
             cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 165, 255), 2) # Orange box
             return frame, current_status

        # --- We have successfully re-acquired the tracked face ---
        (x, y, w, h) = tracked_face
        
        # Update our guess for the next frame
        self.registered_face_center_guess = current_center
        
        current_status = f"Monitoring: {self.registered_person_name} (Active)"
        alert_triggered = False

        # Compare with the last known center
        if self.last_face_center is not None:
            dist_moved = np.linalg.norm(np.array(current_center) - np.array(self.last_face_center))

            if dist_moved < STILLNESS_THRESHOLD_PX:
                # Person is still
                self.stillness_counter += 1
            else:
                # Person moved, reset counter
                self.stillness_counter = 0
        
        # Update the last known center for the *next* frame's comparison
        self.last_face_center = current_center

        # Check if the stillness has crossed the alarm threshold
        if self.stillness_counter > SLEEP_ALARM_FRAMES:
            current_status = f"!!! ALERT: {self.registered_person_name} IS SLEEPING !!!"
            alert_triggered = True
        
        # --- Draw on the frame ---
        if alert_triggered:
            # Draw a bright red box for alert
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 3)
            cv2.putText(frame, f"ALERT: {self.registered_person_name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        else:
            # Draw a standard blue box for the tracked person
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, self.registered_person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(frame, f"Stillness: {self.stillness_counter}/{SLEEP_ALARM_FRAMES}", (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        return frame, current_status


    def detect_sleeping(self, frame):
        """
        This function is no longer called directly by the video_loop.
        Its logic has been moved into process_frame_logic.
        """
        pass # Kept to show what was replaced
        

    def on_closing(self):
        """
        Handles the window close event.
        """
        if messagebox.askokcancel("Quit", "Do you want to exit the application?"):
            self.stop_video_stream()
            self.window.destroy()

# --- Main execution ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog  # Import simpledialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os

# --- Configuration Constants ---

# How many pixels of movement is "not still"?
# This will need tuning based on your camera resolution and distance.
STILLNESS_THRESHOLD_PX = 10

# How many consecutive "still" frames trigger an alarm?
# (e.g., 100 frames at ~20fps is ~5 seconds)
SLEEP_ALARM_FRAMES = 100

# New constant for tracking
# If the closest-found face is further than this from the last frame,
# we assume it's a new person or the tracker lost the original.
MAX_TRACKING_JUMP_PX = 150

class SleepingAlertApp:
    def __init__(self, window):
        """
        Initialize the application.
        """
        self.window = window
        self.window.title("Sleeping Alert System (Dev Phase)")
        self.window.geometry("800x700")

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- Face Detection ---
        # Use the built-in Haar Cascade data from OpenCV
        try:
            face_cascade_path = os.path.join(cv2.data.haarcascades, 'haarcascade_frontalface_alt2.xml') # Using 'alt2' model for better performance
            if not os.path.exists(face_cascade_path):
                raise IOError("Haar cascade file not found.")
            self.face_cascade = cv2.CascadeClassifier(face_cascade_path)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load Haar Cascade: {e}\nPlease ensure OpenCV is correctly installed.")
            self.window.destroy()
            return

        # --- "Sleeping" Logic State ---
        self.last_face_center = None
        self.stillness_counter = 0
        
        # --- New Registration State ---
        self.is_monitoring = False  # Are we actively monitoring a registered person?
        self.registered_face_center_guess = None # The last known center of the monitored person
        self.registered_person_name = None # Store the registered person's name
        self.current_frame_for_registration = None # Temp stores frame for registration
        self.registration_lock = threading.Lock() # Lock for accessing the frame

        # --- GUI Elements ---
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # --- GUI Layout Fix ---
        # Pack non-expanding widgets (buttons, status) to the BOTTOM first.
        # This anchors them to the bottom of the main_frame.
        
        # Control Buttons
        self.button_frame = tk.Frame(self.main_frame)
        self.button_frame.pack(side=tk.BOTTOM, pady=10) # Anchor to bottom

        # Status Label
        self.status_text = tk.StringVar()
        self.status_text.set("Ready. Press 'Start Camera' to begin.")
        self.status_label = tk.Label(self.main_frame, textvariable=self.status_text, font=("Arial", 14))
        self.status_label.pack(side=tk.BOTTOM, pady=5) # Anchor to bottom, above buttons

        # Video Display Label
        # This is packed LAST, so it expands to fill all remaining space.
        self.video_label = tk.Label(self.main_frame, bg="black")
        self.video_label.pack(fill=tk.BOTH, expand=True)

        # --- Original Button/Status placement (removed) ---
        # self.video_label.pack(fill=tk.BOTH, expand=True)
        # self.status_label.pack(pady=10)
        # self.button_frame.pack()
        # --- End of Original Placement ---

        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, font=("Arial", 12), width=15)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, font=("Arial", 12), width=15)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        # --- New Registration Buttons ---
        self.register_button = tk.Button(self.button_frame, text="Register & Monitor", command=self.register_person, font=("Arial", 12), width=18)
        self.register_button.pack(side=tk.LEFT, padx=5)

        self.clear_button = tk.Button(self.button_frame, text="Clear Registration", command=self.clear_registration, font=("Arial", 12), width=18)
        self.clear_button.pack(side=tk.LEFT, padx=5)


        # Set up the close protocol
        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        """
        Starts the video capture in a new thread.
        """
        if self.is_running:
            return

        try:
            self.cap = cv2.VideoCapture(0)  # Use 0 for laptop webcam
            if not self.cap.isOpened():
                raise IOError("Cannot open webcam.")
            
            self.is_running = True
            
            # Start the video processing thread
            # daemon=True ensures the thread will close when the main app closes
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            
            # self.start_button.config(state=tk.DISABLED)
            # self.stop_button.config(state=tk.NORMAL)
            # self.register_button.config(state=tk.NORMAL) # Enable registration
            self.status_text.set("Camera running. Click 'Register & Monitor'.")
        
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            if self.cap:
                self.cap.release()

    def stop_video_stream(self):
        """
        Signals the video loop to stop.
        """
        # Add guard clause
        if not self.is_running:
            return
            
        self.is_running = False
        
        # The thread will see self.is_running is False and exit
        # We wait a moment for the thread to finish
        if self.video_thread:
            self.video_thread.join(timeout=0.5) 
            
        if self.cap:
            self.cap.release()

        # self.start_button.config(state=tk.NORMAL)
        # self.stop_button.config(state=tk.DISABLED)
        self.status_text.set("Camera stopped.")
        self.video_label.config(image=None) # Clear the image
        
        # --- Reset all states ---
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None # Clear name
        self.last_face_center = None
        self.stillness_counter = 0
        # self.register_button.config(state=tk.DISABLED)
        # self.clear_button.config(state=tk.DISABLED)

    def video_loop(self):
        """
        The main loop for video processing. Runs in a separate thread.
        """
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    self.status_text.set("Error: Can't read from camera.")
                    time.sleep(0.5)
                    continue

                # Flip for a "mirror" view, which is more intuitive
                frame = cv2.flip(frame, 1)
                
                # Store a copy of the frame for the registration function
                with self.registration_lock:
                    self.current_frame_for_registration = frame.copy()
                
                # Process the frame
                processed_frame, status = self.process_frame_logic(frame)

                # Update the status text
                self.status_text.set(status)

                # Convert the OpenCV (BGR) frame to a PIL (RGB) image
                cv_img = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                
                # Resize image to fit the label (optional, but good for layout)
                w, h = self.video_label.winfo_width(), self.video_label.winfo_height()
                if w > 1 and h > 1: # Avoid division by zero on init
                     pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)

                # Convert PIL image to Tkinter-compatible image
                imgtk = ImageTk.PhotoImage(image=pil_img)

                # Update the video label in the GUI
                # This must be done from the main thread, but tkinter seems to handle this call
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

            except Exception as e:
                print(f"Error in video loop: {e}")
                self.is_running = False
            
            # Control loop speed slightly
            time.sleep(0.01) # ~100fps theoretical max, but processing will slow it down

        print("Video loop stopped.")

    def register_person(self):
        """
        Registers the largest face currently in the frame for monitoring.
        """
        # --- Add Guard Clauses ---
        if not self.is_running:
            messagebox.showwarning("Error", "Camera is not running. Please start the camera first.")
            return
        
        if self.is_monitoring:
            messagebox.showwarning("Error", "A person is already being monitored. Please clear the registration first.")
            return

        with self.registration_lock:
            if self.current_frame_for_registration is None:
                messagebox.showwarning("Registration Error", "Camera not ready. Please try again.")
                return
            
            # Use the stored frame to find a face
            gray = cv2.cvtColor(self.current_frame_for_registration, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.1,
                minNeighbors=4,  # Tuned parameter (was 5)
                minSize=(40, 40) # Tuned parameter (was 50, 50)
            )

        if len(faces) == 0:
            messagebox.showwarning("Registration Error", "No person detected in the frame. Please face the camera and try again.")
            return

        # Register the largest face
        # We sort by area to be sure we get the main person
        faces_by_area = sorted(faces, key=lambda f: f[2] * f[3], reverse=True)
        (x, y, w, h) = faces_by_area[0]

        center_x = x + w // 2
        center_y = y + h // 2
        
        # --- NEW: Ask for the person's name ---
        name = simpledialog.askstring("Register Person", "Enter the person's name:", parent=self.window)
        
        if not name:
            messagebox.showwarning("Registration Cancelled", "Registration was cancelled (no name provided).")
            return
        
        # --- Lock in the registration ---
        self.is_monitoring = True
        self.registered_person_name = name
        self.registered_face_center_guess = (center_x, center_y)
        self.last_face_center = (center_x, center_y) # Start stillness check from this point
        self.stillness_counter = 0

        # Update GUI
        # self.register_button.config(state=tk.DISABLED)
        # self.clear_button.config(state=tk.NORMAL)
        self.status_text.set(f"Person registered: {self.registered_person_name}. Actively monitoring.")
        messagebox.showinfo("Registration Complete", f"{self.registered_person_name} registered. Monitoring has started.")

    def clear_registration(self):
        """
        Clears the current registration and stops monitoring.
        """
        # --- Add Guard Clause ---
        if not self.is_monitoring:
            messagebox.showwarning("Info", "No person is currently registered.")
            return
            
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None # Clear name
        self.last_face_center = None
        self.stillness_counter = 0

        # Update GUI
        # if self.is_running: # Only enable if camera is on
        #     self.register_button.config(state=tk.NORMAL)
        # self.clear_button.config(state=tk.DISABLED)
        self.status_text.set("Monitoring stopped. Ready to register a new person.")


    def process_frame_logic(self, frame):
        """
        Detects faces and applies monitoring logic based on registration status.
        This function replaces the old 'detect_sleeping'.
        """
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=4,  # Tuned parameter (was 5)
            minSize=(40, 40) # Tuned parameter (was 50, 50)
        )

        # If not monitoring, just show all faces found
        if not self.is_monitoring:
            current_status = "Ready to register."
            if self.is_running: # Check if camera is on
                current_status = "Click 'Register & Monitor' to begin."
            
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2) # Light blue box
            
            return frame, current_status

        # --- If we ARE monitoring ---
        
        if len(faces) == 0:
            # We were monitoring, but now the person is gone
            current_status = f"MONITORING: {self.registered_person_name} lost!"
            self.last_face_center = None # Stop stillness counter
            self.stillness_counter = 0
            return frame, current_status

        # Find the face closest to our last known center
        min_dist = float('inf')
        tracked_face = None
        current_center = None

        for (x, y, w, h) in faces:
            center = (x + w // 2, y + h // 2)
            dist = np.linalg.norm(np.array(center) - np.array(self.registered_face_center_guess))
            
            if dist < min_dist:
                min_dist = dist
                tracked_face = (x, y, w, h)
                current_center = center
        
        # Check if the closest face is "reasonably" close.
        # If not, it's probably a different person.
        if min_dist > MAX_TRACKING_JUMP_PX:
             # The person we found is too far from the last spot.
             current_status = f"MONITORING: {self.registered_person_name} lost! (New face detected)"
             self.last_face_center = None
             self.stillness_counter = 0
             # Draw a box on the *new* face
             (x, y, w, h) = tracked_face
             cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 165, 255), 2) # Orange box
             return frame, current_status

        # --- We have successfully re-acquired the tracked face ---
        (x, y, w, h) = tracked_face
        
        # Update our guess for the next frame
        self.registered_face_center_guess = current_center
        
        current_status = f"Monitoring: {self.registered_person_name} (Active)"
        alert_triggered = False

        # Compare with the last known center
        if self.last_face_center is not None:
            dist_moved = np.linalg.norm(np.array(current_center) - np.array(self.last_face_center))

            if dist_moved < STILLNESS_THRESHOLD_PX:
                # Person is still
                self.stillness_counter += 1
            else:
                # Person moved, reset counter
                self.stillness_counter = 0
        
        # Update the last known center for the *next* frame's comparison
        self.last_face_center = current_center

        # Check if the stillness has crossed the alarm threshold
        if self.stillness_counter > SLEEP_ALARM_FRAMES:
            current_status = f"!!! ALERT: {self.registered_person_name} IS SLEEPING !!!"
            alert_triggered = True
        
        # --- Draw on the frame ---
        if alert_triggered:
            # Draw a bright red box for alert
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 3)
            cv2.putText(frame, f"ALERT: {self.registered_person_name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        else:
            # Draw a standard blue box for the tracked person
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, self.registered_person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(frame, f"Stillness: {self.stillness_counter}/{SLEEP_ALARM_FRAMES}", (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        return frame, current_status


    def detect_sleeping(self, frame):
        """
        This function is no longer called directly by the video_loop.
        Its logic has been moved into process_frame_logic.
        """
        pass # Kept to show what was replaced
        

    def on_closing(self):
        """
        Handles the window close event.
        """
        if messagebox.askokcancel("Quit", "Do you want to exit the application?"):
            self.stop_video_stream()
            self.window.destroy()

# --- Main execution ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os
import mediapipe as mp  # --- Import MediaPipe ---

# --- Configuration Constants ---

# How many pixels of movement is "not still"?
# This will need tuning based on your camera resolution and distance.
STILLNESS_THRESHOLD_PX = 10

# How many consecutive "still" frames trigger an alarm?
# (e.g., 100 frames at ~20fps is ~5 seconds)
SLEEP_ALARM_FRAMES = 100

# New constant for tracking
# If the closest-found face is further than this from the last frame,
# we assume it's a new person or the tracker lost the original.
MAX_TRACKING_JUMP_PX = 150

class SleepingAlertApp:
    def __init__(self, window):
        """
        Initialize the application.
        """
        self.window = window
        self.window.title("Sleeping Alert System (Dev Phase)")
        self.window.geometry("800x700")

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- Face Detection (REPLACED) ---
        # We now use MediaPipe Face Mesh, which is far more accurate.
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=5,  # Detect up to 5 faces
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # --- "Sleeping" Logic State ---
        self.last_face_center = None
        self.stillness_counter = 0
        
        # --- New Registration State ---
        self.is_monitoring = False  # Are we actively monitoring a registered person?
        self.registered_face_center_guess = None # The last known center of the monitored person
        self.registered_person_name = None # Store the registered person's name
        self.current_frame_for_registration = None # Temp stores frame for registration
        self.registration_lock = threading.Lock() # Lock for accessing the frame

        # --- GUI Elements ---
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # --- GUI Layout Fix ---
        # Pack non-expanding widgets (buttons, status) to the BOTTOM first.
        # This anchors them to the bottom of the main_frame.
        
        # Control Buttons
        self.button_frame = tk.Frame(self.main_frame)
        self.button_frame.pack(side=tk.BOTTOM, pady=10) # Anchor to bottom

        # Status Label
        self.status_text = tk.StringVar()
        self.status_text.set("Ready. Press 'Start Camera' to begin.")
        self.status_label = tk.Label(self.main_frame, textvariable=self.status_text, font=("Arial", 14))
        self.status_label.pack(side=tk.BOTTOM, pady=5) # Anchor to bottom, above buttons

        # Video Display Label
        # This is packed LAST, so it expands to fill all remaining space.
        self.video_label = tk.Label(self.main_frame, bg="black")
        self.video_label.pack(fill=tk.BOTH, expand=True)

        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, font=("Arial", 12), width=15)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, font=("Arial", 12), width=15)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        # --- New Registration Buttons ---
        self.register_button = tk.Button(self.button_frame, text="Register & Monitor", command=self.register_person, font=("Arial", 12), width=18)
        self.register_button.pack(side=tk.LEFT, padx=5)

        self.clear_button = tk.Button(self.button_frame, text="Clear Registration", command=self.clear_registration, font=("Arial", 12), width=18)
        self.clear_button.pack(side=tk.LEFT, padx=5)


        # Set up the close protocol
        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        """
        Starts the video capture in a new thread.
        """
        if self.is_running:
            return

        try:
            self.cap = cv2.VideoCapture(0)  # Use 0 for laptop webcam
            if not self.cap.isOpened():
                raise IOError("Cannot open webcam.")
            
            self.is_running = True
            
            # Start the video processing thread
            # daemon=True ensures the thread will close when the main app closes
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            
            self.status_text.set("Camera running. Click 'Register & Monitor'.")
        
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            if self.cap:
                self.cap.release()

    def stop_video_stream(self):
        """
        Signals the video loop to stop.
        """
        # Add guard clause
        if not self.is_running:
            return
            
        self.is_running = False
        
        # The thread will see self.is_running is False and exit
        # We wait a moment for the thread to finish
        if self.video_thread:
            self.video_thread.join(timeout=0.5) 
            
        if self.cap:
            self.cap.release()

        self.status_text.set("Camera stopped.")
        self.video_label.config(image=None) # Clear the image
        
        # --- Reset all states ---
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None # Clear name
        self.last_face_center = None
        self.stillness_counter = 0

    def video_loop(self):
        """
        The main loop for video processing. Runs in a separate thread.
        """
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    self.status_text.set("Error: Can't read from camera.")
                    time.sleep(0.5)
                    continue

                # Flip for a "mirror" view, which is more intuitive
                frame = cv2.flip(frame, 1)
                
                # Store a copy of the frame for the registration function
                with self.registration_lock:
                    self.current_frame_for_registration = frame.copy()
                
                # --- FIX: All logic below was moved inside the try block ---

                # MediaPipe needs RGB, but OpenCV provides BGR.
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Process the frame
                processed_frame, status = self.process_frame_logic(frame, rgb_frame) # Pass rgb_frame

                # Update the status text
                self.status_text.set(status)

                # Convert the OpenCV (BGR) frame to a PIL (RGB) image
                cv_img = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                
                # Resize image to fit the label (optional, but good for layout)
                w, h = self.video_label.winfo_width(), self.video_label.winfo_height()
                if w > 1 and h > 1: # Avoid division by zero on init
                     pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)

                # Convert PIL image to Tkinter-compatible image
                imgtk = ImageTk.PhotoImage(image=pil_img)

                # Update the video label in the GUI
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

            except Exception as e:
                print(f"Error in video loop: {e}")
                self.is_running = False
            
            # Control loop speed slightly
            time.sleep(0.01) # ~100fps theoretical max, but processing will slow it down

        print("Video loop stopped.")

    # --- Helper function to get face data from MediaPipe results ---
    # --- FIX: This function was completely rebuilt from the corrupted version ---
    def get_faces_from_results(self, frame, results):
        """
        Extracts bounding boxes and nose-tip centers from MediaPipe results.
        """
        faces_data = []
        h, w, _ = frame.shape

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # --- 1. Get Bounding Box ---
                x_min, y_min = w, h
                x_max, y_max = 0, 0
                for landmark in face_landmarks.landmark:
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    if x < x_min: x_min = x
                    if y < y_min: y_min = y
                    if x > x_max: x_max = x
                    if y > y_max: y_max = y
                
                # Add a little padding
                x_min = max(0, x_min - 10)
                y_min = max(0, y_min - 10)
                x_max = min(w, x_max + 10)
                y_max = min(h, y_max + 10)

                box = (x_min, y_min, x_max - x_min, y_max - y_min)

                # --- 2. Get Nose Tip (Landmark 1) for precise tracking ---
                # This is our new "center" for movement tracking
                nose_tip = face_landmarks.landmark[1]
                center = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                faces_data.append({'box': box, 'center': center})
        
        return faces_data

    def register_person(self):
        """
        Registers the largest face currently in the frame for monitoring.
        """
        # --- FIX: Removed corrupted code lines from here ---

        with self.registration_lock:
            if self.current_frame_for_registration is None:
                messagebox.showwarning("Registration Error", "Camera not ready. Please try again.")
                return
            
            # Use the stored frame to find a face
            # MediaPipe needs RGB
            rgb_frame_reg = cv2.cvtColor(self.current_frame_for_registration, cv2.COLOR_BGR2RGB)
            results = self.face_mesh.process(rgb_frame_reg)
            faces = self.get_faces_from_results(self.current_frame_for_registration, results) # Call self.get_faces...

        if len(faces) == 0:
            messagebox.showwarning("Registration Error", "No person detected in the frame. Please face the camera and try again.")
            return

        # Register the largest face
        # faces list is now a list of dicts: [{'box': (x,y,w,h), 'center': (cx,cy)}, ...]
        faces_by_area = sorted(faces, key=lambda f: f['box'][2] * f['box'][3], reverse=True)
        largest_face = faces_by_area[0]

        (x, y, w, h) = largest_face['box']
        center_x, center_y = largest_face['center']
        
        # --- NEW: Ask for the person's name ---
        name = simpledialog.askstring("Register Person", "Enter the person's name:", parent=self.window)
        
        if not name:
            messagebox.showwarning("Registration Cancelled", "Registration was cancelled (no name provided).")
            return
        
        # --- Lock in the registration ---
        self.is_monitoring = True
        self.registered_person_name = name
        self.registered_face_center_guess = (center_x, center_y)
        self.last_face_center = (center_x, center_y) # Start stillness check from this point
        self.stillness_counter = 0

        # Update GUI
        self.status_text.set(f"Person registered: {self.registered_person_name}. Actively monitoring.")
        messagebox.showinfo("Registration Complete", f"{self.registered_person_name} registered. Monitoring has started.")

    def clear_registration(self):
        """
        Clears the current registration and stops monitoring.
        """
        # --- Add Guard Clause ---
        if not self.is_monitoring:
            messagebox.showwarning("Info", "No person is currently registered.")
            return
            
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None # Clear name
        self.last_face_center = None
        self.stillness_counter = 0

        self.status_text.set("Monitoring stopped. Ready to register a new person.")


    def process_frame_logic(self, frame, rgb_frame):
        """
        Detects faces and applies monitoring logic based on registration status.
        This function replaces the old 'detect_sleeping'.
        'frame' is BGR (for drawing), 'rgb_frame' is for MediaPipe.
        """
        # Process the frame with MediaPipe
        results = self.face_mesh.process(rgb_frame)
        
        # Get face data
        faces = self.get_faces_from_results(frame, results) # Call self.get_faces...

        # If not monitoring, just show all faces found
        if not self.is_monitoring:
            current_status = "Ready to register."
            if self.is_running: # Check if camera is on
                current_status = "Click 'Register & Monitor' to begin."
            
            for face_data in faces:
                (x, y, w, h) = face_data['box']
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2) # Light blue box
            
            return frame, current_status

        # --- If we ARE monitoring ---
        
        if len(faces) == 0:
            # We were monitoring, but now the person is gone
            current_status = f"MONITORING: {self.registered_person_name} lost!"
            self.last_face_center = None # Stop stillness counter
            self.stillness_counter = 0
            return frame, current_status

        # Find the face closest to our last known center
        min_dist = float('inf')
        tracked_face_data = None
        current_center = None # This will be the nose tip

        for face_data in faces:
            center = face_data['center']
            dist = np.linalg.norm(np.array(center) - np.array(self.registered_face_center_guess))
            
            if dist < min_dist:
                min_dist = dist
                tracked_face_data = face_data
                current_center = center
        
        # Check if the closest face is "reasonably" close.
        # If not, it's probably a different person.
        if min_dist > MAX_TRACKING_JUMP_PX:
             # The person we found is too far from the last spot.
             current_status = f"MONITORING: {self.registered_person_name} lost! (New face detected)"
             self.last_face_center = None
             self.stillness_counter = 0
             # Draw a box on the *new* face
             (x, y, w, h) = tracked_face_data['box']
             cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 165, 255), 2) # Orange box
             return frame, current_status

        # --- We have successfully re-acquired the tracked face ---
        (x, y, w, h) = tracked_face_data['box']
        
        # Update our guess for the next frame
        self.registered_face_center_guess = current_center
        
        current_status = f"Monitoring: {self.registered_person_name} (Active)"
        alert_triggered = False

        # Compare with the last known center
        if self.last_face_center is not None:
            dist_moved = np.linalg.norm(np.array(current_center) - np.array(self.last_face_center))

            if dist_moved < STILLNESS_THRESHOLD_PX:
                # Person is still
                self.stillness_counter += 1
            else:
                # Person moved, reset counter
                self.stillness_counter = 0
        
        # Update the last known center for the *next* frame's comparison
        self.last_face_center = current_center

        # Check if the stillness has crossed the alarm threshold
        if self.stillness_counter > SLEEP_ALARM_FRAMES:
            current_status = f"!!! ALERT: {self.registered_person_name} IS SLEEPING !!!"
            alert_triggered = True
        
        # --- Draw on the frame ---
        if alert_triggered:
            # Draw a bright red box for alert
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 3)
            cv2.putText(frame, f"ALERT: {self.registered_person_name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        else:
            # Draw a standard blue box for the tracked person
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, self.registered_person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            cv2.putText(frame, f"Stillness: {self.stillness_counter}/{SLEEP_ALARM_FRAMES}", (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

        return frame, current_status


    def detect_sleeping(self, frame):
        """
        This function is no longer called directly by the video_loop.
        Its logic has been moved into process_frame_logic.
        """
        pass # Kept to show what was replaced
        

    def on_closing(self):
        """
        Handles the window close event.
        """
        if messagebox.askokcancel("Quit", "Do you want to exit the application?"):
            self.stop_video_stream()
            self.window.destroy()

# --- Main execution ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except Exception as e:
        print(f"An error occurred: {e}")

In [3]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os
import mediapipe as mp  # --- Import MediaPipe ---

# --- Configuration Constants ---

# --- New Eye Aspect Ratio (EAR) Constants ---
# Threshold for "closed" eyes. You will need to tune this value.
# Start with 0.20. Lower it if it triggers when blinking.
EYE_AR_THRESHOLD = 0.20

# How many consecutive frames of "closed" eyes trigger an alarm?
# User requested 10 seconds. Assuming ~20fps, 10 * 20 = 200 frames.
EYE_AR_CONSEC_FRAMES = 80


# New constant for tracking
# If the closest-found face is further than this from the last frame,
# we assume it's a new person or the tracker lost the original.
MAX_TRACKING_JUMP_PX = 150

class SleepingAlertApp:
    def __init__(self, window):
        """
        Initialize the application.
        """
        self.window = window
        self.window.title("Sleeping Alert System (Dev Phase)")
        self.window.geometry("800x700")

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- Face Detection (REPLACED) ---
        # We now use MediaPipe Face Mesh, which is far more accurate.
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=5,  # Detect up to 5 faces
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # --- "Sleeping" Logic State ---
        self.eye_closed_counter = 0 # Replaces stillness_counter
        
        # --- New Registration State ---
        self.is_monitoring = False  # Are we actively monitoring a registered person?
        self.registered_face_center_guess = None # The last known center of the monitored person
        self.registered_person_name = None # Store the registered person's name
        self.current_frame_for_registration = None # Temp stores frame for registration
        self.registration_lock = threading.Lock() # Lock for accessing the frame

        # --- GUI Elements ---
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # --- GUI Layout Fix ---
        # Pack non-expanding widgets (buttons, status) to the BOTTOM first.
        # This anchors them to the bottom of the main_frame.
        
        # Control Buttons
        self.button_frame = tk.Frame(self.main_frame)
        self.button_frame.pack(side=tk.BOTTOM, pady=10) # Anchor to bottom

        # Status Label
        self.status_text = tk.StringVar()
        self.status_text.set("Ready. Press 'Start Camera' to begin.")
        self.status_label = tk.Label(self.main_frame, textvariable=self.status_text, font=("Arial", 14))
        self.status_label.pack(side=tk.BOTTOM, pady=5) # Anchor to bottom, above buttons

        # Video Display Label
        # This is packed LAST, so it expands to fill all remaining space.
        self.video_label = tk.Label(self.main_frame, bg="black")
        self.video_label.pack(fill=tk.BOTH, expand=True)

        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, font=("Arial", 12), width=15)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, font=("Arial", 12), width=15)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        # --- New Registration Buttons ---
        self.register_button = tk.Button(self.button_frame, text="Register & Monitor", command=self.register_person, font=("Arial", 12), width=18)
        self.register_button.pack(side=tk.LEFT, padx=5)

        self.clear_button = tk.Button(self.button_frame, text="Clear Registration", command=self.clear_registration, font=("Arial", 12), width=18)
        self.clear_button.pack(side=tk.LEFT, padx=5)


        # Set up the close protocol
        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        """
        Starts the video capture in a new thread.
        """
        if self.is_running:
            return

        try:
            self.cap = cv2.VideoCapture(0)  # Use 0 for laptop webcam
            if not self.cap.isOpened():
                raise IOError("Cannot open webcam.")
            
            self.is_running = True
            
            # Start the video processing thread
            # daemon=True ensures the thread will close when the main app closes
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            
            self.status_text.set("Camera running. Click 'Register & Monitor'.")
        
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            if self.cap:
                self.cap.release()

    def stop_video_stream(self):
        """
        Signals the video loop to stop.
        """
        # Add guard clause
        if not self.is_running:
            return
            
        self.is_running = False
        
        # The thread will see self.is_running is False and exit
        # We wait a moment for the thread to finish
        if self.video_thread:
            self.video_thread.join(timeout=0.5) 
            
        if self.cap:
            self.cap.release()

        self.status_text.set("Camera stopped.")
        self.video_label.config(image=None) # Clear the image
        
        # --- Reset all states ---
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None # Clear name
        self.eye_closed_counter = 0

    def video_loop(self):
        """
        The main loop for video processing. Runs in a separate thread.
        """
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    self.status_text.set("Error: Can't read from camera.")
                    time.sleep(0.5)
                    continue

                # Flip for a "mirror" view, which is more intuitive
                frame = cv2.flip(frame, 1)
                
                # Store a copy of the frame for the registration function
                with self.registration_lock:
                    self.current_frame_for_registration = frame.copy()
                
                # MediaPipe needs RGB, but OpenCV provides BGR.
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Process the frame
                processed_frame, status = self.process_frame_logic(frame, rgb_frame) # Pass rgb_frame

                # Update the status text
                self.status_text.set(status)

                # Convert the OpenCV (BGR) frame to a PIL (RGB) image
                cv_img = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                
                # Resize image to fit the label (optional, but good for layout)
                w, h = self.video_label.winfo_width(), self.video_label.winfo_height()
                if w > 1 and h > 1: # Avoid division by zero on init
                     pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)

                # Convert PIL image to Tkinter-compatible image
                imgtk = ImageTk.PhotoImage(image=pil_img)

                # Update the video label in the GUI
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

            except Exception as e:
                print(f"Error in video loop: {e}")
                self.is_running = False
            
            # Control loop speed slightly
            time.sleep(0.01) # ~100fps theoretical max, but processing will slow it down

        print("Video loop stopped.")

    # --- NEW HELPER: Calculate Eye Aspect Ratio (EAR) ---
    def get_ear(self, eye_points, w, h):
        """
        Calculates the Eye Aspect Ratio (EAR) given 6 landmark points.
        The points should be in the order: [P1, P4, P2, P6, P3, P5]
        """
        try:
            # Convert landmarks to (x, y) tuples
            p1 = (int(eye_points[0].x * w), int(eye_points[0].y * h))
            p4 = (int(eye_points[1].x * w), int(eye_points[1].y * h))
            p2 = (int(eye_points[2].x * w), int(eye_points[2].y * h))
            p6 = (int(eye_points[3].x * w), int(eye_points[3].y * h))
            p3 = (int(eye_points[4].x * w), int(eye_points[4].y * h))
            p5 = (int(eye_points[5].x * w), int(eye_points[5].y * h))

            # Helper to calculate euclidean distance
            def get_dist(p_a, p_b):
                return np.linalg.norm(np.array(p_a) - np.array(p_b))

            # Vertical distances
            v_dist_1 = get_dist(p2, p6)
            v_dist_2 = get_dist(p3, p5)

            # Horizontal distance
            h_dist = get_dist(p1, p4)

            # Avoid division by zero
            if h_dist == 0:
                return 0.3 # Return an "open" value

            # Calculate EAR
            ear = (v_dist_1 + v_dist_2) / (2.0 * h_dist)
            return ear
        except Exception as e:
            print(f"Error calculating EAR: {e}")
            return 0.3 # Default to "open"

    # --- Helper function to get face data from MediaPipe results ---
    def get_faces_from_results(self, frame, results):
        """
        Extracts bounding boxes and nose-tip centers from MediaPipe results.
        """
        faces_data = []
        h, w, _ = frame.shape

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # --- 1. Get Bounding Box ---
                x_min, y_min = w, h
                x_max, y_max = 0, 0
                for landmark in face_landmarks.landmark:
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    if x < x_min: x_min = x
                    if y < y_min: y_min = y
                    if x > x_max: x_max = x
                    if y > y_max: y_max = y
                
                # Add a little padding
                x_min = max(0, x_min - 10)
                y_min = max(0, y_min - 10)
                x_max = min(w, x_max + 10)
                y_max = min(h, y_max + 10)

                box = (x_min, y_min, x_max - x_min, y_max - y_min)

                # --- 2. Get Nose Tip (Landmark 1) for precise tracking ---
                # This is our new "center" for movement tracking
                nose_tip = face_landmarks.landmark[1]
                center = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # --- 3. Calculate Eye Aspect Ratio (EAR) ---
                # These are the specific landmark indices for the EAR calculation
                RIGHT_EYE_EAR_POINTS_INDICES = [33, 133, 159, 145, 158, 153] # P1, P4, P2, P6, P3, P5
                LEFT_EYE_EAR_POINTS_INDICES = [362, 263, 386, 374, 385, 380] # P1, P4, P2, P6, P3, P5

                landmarks = face_landmarks.landmark
                
                right_eye_points = [landmarks[i] for i in RIGHT_EYE_EAR_POINTS_INDICES]
                left_eye_points = [landmarks[i] for i in LEFT_EYE_EAR_POINTS_INDICES]

                right_ear = self.get_ear(right_eye_points, w, h)
                left_ear = self.get_ear(left_eye_points, w, h)
                
                avg_ear = (left_ear + right_ear) / 2.0
                
                faces_data.append({'box': box, 'center': center, 'avg_ear': avg_ear})
        
        return faces_data

    def register_person(self):
        """
        Registers the largest face currently in the frame for monitoring.
        """
        with self.registration_lock:
            if self.current_frame_for_registration is None:
                messagebox.showwarning("Registration Error", "Camera not ready. Please try again.")
                return
            
            # Use the stored frame to find a face
            # MediaPipe needs RGB
            rgb_frame_reg = cv2.cvtColor(self.current_frame_for_registration, cv2.COLOR_BGR2RGB)
            results = self.face_mesh.process(rgb_frame_reg)
            faces = self.get_faces_from_results(self.current_frame_for_registration, results) # Call self.get_faces...

        if len(faces) == 0:
            messagebox.showwarning("Registration Error", "No person detected in the frame. Please face the camera and try again.")
            return

        # Register the largest face
        # faces list is now a list of dicts: [{'box': (x,y,w,h), 'center': (cx,cy)}, ...]
        faces_by_area = sorted(faces, key=lambda f: f['box'][2] * f['box'][3], reverse=True)
        largest_face = faces_by_area[0]

        (x, y, w, h) = largest_face['box']
        center_x, center_y = largest_face['center']
        
        # --- NEW: Ask for the person's name ---
        name = simpledialog.askstring("Register Person", "Enter the person's name:", parent=self.window)
        
        if not name:
            messagebox.showwarning("Registration Cancelled", "Registration was cancelled (no name provided).")
            return
        
        # --- Lock in the registration ---
        self.is_monitoring = True
        self.registered_person_name = name
        self.registered_face_center_guess = (center_x, center_y)
        self.eye_closed_counter = 0

        # Update GUI
        self.status_text.set(f"Person registered: {self.registered_person_name}. Actively monitoring.")
        messagebox.showinfo("Registration Complete", f"{self.registered_person_name} registered. Monitoring has started.")

    def clear_registration(self):
        """
        Clears the current registration and stops monitoring.
        """
        # --- Add Guard Clause ---
        if not self.is_monitoring:
            messagebox.showwarning("Info", "No person is currently registered.")
            return
            
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None # Clear name
        self.eye_closed_counter = 0

        self.status_text.set("Monitoring stopped. Ready to register a new person.")


    def process_frame_logic(self, frame, rgb_frame):
        """
        Detects faces and applies monitoring logic based on registration status.
        This function replaces the old 'detect_sleeping'.
        'frame' is BGR (for drawing), 'rgb_frame' is for MediaPipe.
        """
        # Process the frame with MediaPipe
        results = self.face_mesh.process(rgb_frame)
        
        # Get face data
        faces = self.get_faces_from_results(frame, results) # Call self.get_faces...

        # If not monitoring, just show all faces found
        if not self.is_monitoring:
            current_status = "Ready to register."
            if self.is_running: # Check if camera is on
                current_status = "Click 'Register & Monitor' to begin."
            
            for face_data in faces:
                (x, y, w, h) = face_data['box']
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2) # Light blue box
            
            return frame, current_status

        # --- If we ARE monitoring ---
        
        if len(faces) == 0:
            # We were monitoring, but now the person is gone
            current_status = f"MONITORING: {self.registered_person_name} lost!"
            self.eye_closed_counter = 0
            return frame, current_status

        # Find the face closest to our last known center
        min_dist = float('inf')
        tracked_face_data = None
        current_center = None # This will be the nose tip

        for face_data in faces:
            center = face_data['center']
            dist = np.linalg.norm(np.array(center) - np.array(self.registered_face_center_guess))
            
            if dist < min_dist:
                min_dist = dist
                tracked_face_data = face_data
                current_center = center
        
        # Check if the closest face is "reasonably" close.
        # If not, it's probably a different person.
        if min_dist > MAX_TRACKING_JUMP_PX:
             # The person we found is too far from the last spot.
             current_status = f"MONITORING: {self.registered_person_name} lost! (New face detected)"
             self.eye_closed_counter = 0
             # Draw a box on the *new* face
             (x, y, w, h) = tracked_face_data['box']
             cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 165, 255), 2) # Orange box
             return frame, current_status

        # --- We have successfully re-acquired the tracked face ---
        (x, y, w, h) = tracked_face_data['box']
        
        # Update our guess for the next frame
        self.registered_face_center_guess = current_center
        
        current_status = f"Monitoring: {self.registered_person_name} (Active)"
        alert_triggered = False
        
        # --- Get the EAR value for the tracked face ---
        avg_ear = tracked_face_data['avg_ear']

        # --- NEW: Eye Aspect Ratio (EAR) Logic ---
        if avg_ear < EYE_AR_THRESHOLD:
            # Eyes are closed
            self.eye_closed_counter += 1
        else:
            # Eyes are open, reset counter
            self.eye_closed_counter = 0
        
        # Check if the "eyes closed" counter has crossed the alarm threshold
        if self.eye_closed_counter > EYE_AR_CONSEC_FRAMES:
            current_status = f"!!! ALERT: {self.registered_person_name} EYES CLOSED !!!"
            alert_triggered = True
        else:
            # Update status text if not alerting
            current_status = f"Monitoring: {self.registered_person_name} (Eyes Open)"
        
        # --- Draw on the frame ---
        if alert_triggered:
            # Draw a bright red box for alert
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 3)
            cv2.putText(frame, f"ALERT: {self.registered_person_name} (EYES CLOSED)", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            # Draw a standard blue box for the tracked person
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, self.registered_person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            
            # --- MODIFIED: Show eye counter instead of stillness ---
            cv2.putText(frame, f"Eye Closed Count: {self.eye_closed_counter}/{EYE_AR_CONSEC_FRAMES}", (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
            
            # --- ADDED: Show live EAR value for tuning ---
            cv2.putText(frame, f"EAR: {avg_ear:.2f}", (x, y + h + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        return frame, current_status


    def detect_sleeping(self, frame):
        """
        This function is no longer called directly by the video_loop.
        Its logic has been moved into process_frame_logic.
        """
        pass # Kept to show what was replaced
        

    def on_closing(self):
        """
        Handles the window close event.
        """
        if messagebox.askokcancel("Quit", "Do you want to exit the application?"):
            self.stop_video_stream()
            self.window.destroy()

# --- Main execution ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except Exception as e:
        print(f"An error occurred: {e}")

Error in video loop: main thread is not in main loop
Video loop stopped.


Exception ignored in: <function Variable.__del__ at 0x00000238D8AD58A0>
Traceback (most recent call last):
  File "E:\Application\Python3114\Lib\tkinter\__init__.py", line 410, in __del__
    if self._tk.getboolean(self._tk.call("info", "exists", self._name)):
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: main thread is not in main loop


In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os
import mediapipe as mp  # --- Import MediaPipe ---

# --- Configuration Constants ---

# --- New Dynamic Calibration Constants ---
# We no longer use a fixed EAR threshold.
# Instead, we set a personal threshold as a percentage of the user's open-eye value.
# e.g., if open EAR is 0.30, threshold will be 0.30 * 0.75 = 0.225
THRESHOLD_CALIBRATION_FACTOR = 0.75

# Sanity check: if the user's "open" EAR is below this,
# they probably tried to register with their eyes closed.
MIN_OPEN_EYE_EAR = 0.20

# How many consecutive frames of "closed" eyes trigger an alarm?
# User requested 10 seconds. Assuming ~20fps, 10 * 20 = 200 frames.
EYE_AR_CONSEC_FRAMES = 200

# Tracking constant: max distance a face can "jump" between frames.
MAX_TRACKING_JUMP_PX = 150

class SleepingAlertApp:
    def __init__(self, window):
        """
        Initialize the application.
        """
        self.window = window
        self.window.title("Sleeping Alert System (Calibrated)")
        self.window.geometry("800x700")

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- MediaPipe Face Mesh Initialization ---
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=5,  # Detect up to 5 faces
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # --- Logic & Registration State ---
        self.eye_closed_counter = 0
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None
        self.current_frame_for_registration = None
        self.registration_lock = threading.Lock()

        # --- New Calibration State Variables ---
        # These are set during registration and are unique to the user.
        self.calibrated_open_ear = 0.0
        self.calibrated_threshold = 0.0

        # --- GUI Elements ---
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # --- GUI Layout Fix: Anchor controls to BOTTOM ---
        
        self.button_frame = tk.Frame(self.main_frame)
        self.button_frame.pack(side=tk.BOTTOM, pady=10)

        self.status_text = tk.StringVar()
        self.status_text.set("Ready. Press 'Start Camera' to begin.")
        self.status_label = tk.Label(self.main_frame, textvariable=self.status_text, font=("Arial", 14))
        self.status_label.pack(side=tk.BOTTOM, pady=5)

        # Video label expands to fill the remaining space
        self.video_label = tk.Label(self.main_frame, bg="black")
        self.video_label.pack(fill=tk.BOTH, expand=True)

        # --- Buttons ---
        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, font=("Arial", 12), width=15)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, font=("Arial", 12), width=15)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        self.register_button = tk.Button(self.button_frame, text="Register & Monitor", command=self.register_person, font=("Arial", 12), width=18)
        self.register_button.pack(side=tk.LEFT, padx=5)

        self.clear_button = tk.Button(self.button_frame, text="Clear Registration", command=self.clear_registration, font=("Arial", 12), width=18)
        self.clear_button.pack(side=tk.LEFT, padx=5)

        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        """
        Starts the video capture in a new thread.
        """
        if self.is_running:
            return

        try:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                raise IOError("Cannot open webcam.")
            
            self.is_running = True
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            self.status_text.set("Camera running. Click 'Register & Monitor'.")
        
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            if self.cap:
                self.cap.release()

    def stop_video_stream(self):
        """
        Signals the video loop to stop and resets all states.
        """
        if not self.is_running:
            return
            
        self.is_running = False
        if self.video_thread:
            self.video_thread.join(timeout=0.5) 
            
        if self.cap:
            self.cap.release()

        self.video_label.config(image=None)
        self.status_text.set("Camera stopped.")
        
        # Reset all logic states
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None
        self.eye_closed_counter = 0
        self.calibrated_open_ear = 0.0
        self.calibrated_threshold = 0.0

    def video_loop(self):
        """
        The main loop for video processing. Runs in a separate thread.
        """
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    self.status_text.set("Error: Can't read from camera.")
                    time.sleep(0.5)
                    continue

                frame = cv2.flip(frame, 1)
                
                with self.registration_lock:
                    self.current_frame_for_registration = frame.copy()
                
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Process the frame and get status
                processed_frame, status = self.process_frame_logic(frame, rgb_frame)

                self.status_text.set(status)

                # Convert for Tkinter display
                cv_img = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                
                w, h = self.video_label.winfo_width(), self.video_label.winfo_height()
                if w > 1 and h > 1:
                     pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)

                imgtk = ImageTk.PhotoImage(image=pil_img)
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

            except Exception as e:
                print(f"Error in video loop: {e}")
                self.is_running = False
            
            time.sleep(0.01)

        print("Video loop stopped.")

    def get_ear(self, eye_points, w, h):
        """
        Calculates the Eye Aspect Ratio (EAR) given 6 landmark points.
        """
        try:
            # Convert landmarks to (x, y) tuples
            p1 = (int(eye_points[0].x * w), int(eye_points[0].y * h))
            p4 = (int(eye_points[1].x * w), int(eye_points[1].y * h))
            p2 = (int(eye_points[2].x * w), int(eye_points[2].y * h))
            p6 = (int(eye_points[3].x * w), int(eye_points[3].y * h))
            p3 = (int(eye_points[4].x * w), int(eye_points[4].y * h))
            p5 = (int(eye_points[5].x * w), int(eye_points[5].y * h))

            def get_dist(p_a, p_b):
                return np.linalg.norm(np.array(p_a) - np.array(p_b))

            v_dist_1 = get_dist(p2, p6)
            v_dist_2 = get_dist(p3, p5)
            h_dist = get_dist(p1, p4)

            if h_dist == 0:
                return 0.3 # Default "open"

            ear = (v_dist_1 + v_dist_2) / (2.0 * h_dist)
            return ear
        except Exception:
            return 0.3 # Default "open"

    def get_faces_from_results(self, frame, results):
        """
        Extracts face data (box, center, EAR) from MediaPipe results.
        """
        faces_data = []
        h, w, _ = frame.shape

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # 1. Get Bounding Box
                x_min, y_min, x_max, y_max = w, h, 0, 0
                for landmark in face_landmarks.landmark:
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    if x < x_min: x_min = x
                    if y < y_min: y_min = y
                    if x > x_max: x_max = x
                    if y > y_max: y_max = y
                
                padding = 10
                x_min = max(0, x_min - padding)
                y_min = max(0, y_min - padding)
                x_max = min(w, x_max + padding)
                y_max = min(h, y_max + padding)
                box = (x_min, y_min, x_max - x_min, y_max - y_min)

                # 2. Get Nose Tip (Landmark 1) for tracking
                nose_tip = face_landmarks.landmark[1]
                center = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 3. Calculate Eye Aspect Ratio (EAR)
                RIGHT_EYE_EAR_POINTS_INDICES = [33, 133, 159, 145, 158, 153]
                LEFT_EYE_EAR_POINTS_INDICES = [362, 263, 386, 374, 385, 380]

                landmarks = face_landmarks.landmark
                right_eye_points = [landmarks[i] for i in RIGHT_EYE_EAR_POINTS_INDICES]
                left_eye_points = [landmarks[i] for i in LEFT_EYE_EAR_POINTS_INDICES]

                right_ear = self.get_ear(right_eye_points, w, h)
                left_ear = self.get_ear(left_eye_points, w, h)
                avg_ear = (left_ear + right_ear) / 2.0
                
                faces_data.append({'box': box, 'center': center, 'avg_ear': avg_ear})
        
        return faces_data

    def register_person(self):
        """
        Registers a person and dynamically calibrates the EAR threshold.
        """
        with self.registration_lock:
            if self.current_frame_for_registration is None:
                messagebox.showwarning("Registration Error", "Camera not ready. Please try again.")
                return
            
            rgb_frame_reg = cv2.cvtColor(self.current_frame_for_registration, cv2.COLOR_BGR2RGB)
            results = self.face_mesh.process(rgb_frame_reg)
            faces = self.get_faces_from_results(self.current_frame_for_registration, results)

        if len(faces) == 0:
            messagebox.showwarning("Registration Error", "No person detected. Please face the camera and try again.")
            return

        faces_by_area = sorted(faces, key=lambda f: f['box'][2] * f['box'][3], reverse=True)
        largest_face = faces_by_area[0]

        # --- DYNAMIC CALIBRATION ---
        current_ear = largest_face['avg_ear']
        
        if current_ear < MIN_OPEN_EYE_EAR:
            messagebox.showwarning("Registration Error", f"Eyes seem to be closed (EAR: {current_ear:.2f}).\nPlease open your eyes and try again.")
            return

        self.calibrated_open_ear = current_ear
        self.calibrated_threshold = current_ear * THRESHOLD_CALIBRATION_FACTOR
        
        name = simpledialog.askstring("Register Person", "Enter the person's name:", parent=self.window)
        if not name:
            messagebox.showwarning("Registration Cancelled", "Registration was cancelled.")
            return
        
        # --- Lock in the registration ---
        (x, y, w, h) = largest_face['box']
        self.is_monitoring = True
        self.registered_person_name = name
        self.registered_face_center_guess = largest_face['center']
        self.eye_closed_counter = 0

        status = f"Calibrated for {name}. Open EAR: {self.calibrated_open_ear:.2f}, Threshold: {self.calibrated_threshold:.2f}"
        self.status_text.set(status)
        messagebox.showinfo("Registration Complete", status)

    def clear_registration(self):
        """
        Clears the current registration and stops monitoring.
        """
        if not self.is_monitoring:
            messagebox.showwarning("Info", "No person is currently registered.")
            return
        
        # Reset all logic states
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None
        self.eye_closed_counter = 0
        self.calibrated_open_ear = 0.0
        self.calibrated_threshold = 0.0

        self.status_text.set("Monitoring stopped. Ready to register a new person.")


    def process_frame_logic(self, frame, rgb_frame):
        """
        Main logic: finds faces, tracks registered person, checks EAR.
        """
        results = self.face_mesh.process(rgb_frame)
        faces = self.get_faces_from_results(frame, results)

        # If not monitoring, just show all faces
        if not self.is_monitoring:
            current_status = "Click 'Register & Monitor' to begin."
            for face_data in faces:
                (x, y, w, h) = face_data['box']
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2)
            return frame, current_status

        # --- If we ARE monitoring ---
        
        if len(faces) == 0:
            current_status = f"MONITORING: {self.registered_person_name} lost!"
            self.eye_closed_counter = 0
            return frame, current_status

        # Find the face closest to our last known center
        min_dist = float('inf')
        tracked_face_data = None
        current_center = None

        for face_data in faces:
            center = face_data['center']
            dist = np.linalg.norm(np.array(center) - np.array(self.registered_face_center_guess))
            if dist < min_dist:
                min_dist = dist
                tracked_face_data = face_data
                current_center = center
        
        if min_dist > MAX_TRACKING_JUMP_PX:
             current_status = f"MONITORING: {self.registered_person_name} lost! (New face detected)"
             self.eye_closed_counter = 0
             (x, y, w, h) = tracked_face_data['box']
             cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 165, 255), 2)
             return frame, current_status

        # --- We have re-acquired the tracked face ---
        (x, y, w, h) = tracked_face_data['box']
        self.registered_face_center_guess = current_center
        avg_ear = tracked_face_data['avg_ear']
        alert_triggered = False
        
        # --- CALIBRATED Eye Aspect Ratio (EAR) Logic ---
        if avg_ear < self.calibrated_threshold:
            self.eye_closed_counter += 1
        else:
            self.eye_closed_counter = 0
        
        if self.eye_closed_counter > EYE_AR_CONSEC_FRAMES:
            current_status = f"!!! ALERT: {self.registered_person_name} EYES CLOSED !!!"
            alert_triggered = True
        else:
            current_status = f"Monitoring: {self.registered_person_name} (Eyes Open)"
        
        # --- Draw on the frame ---
        if alert_triggered:
            color = (0, 0, 255) # Red for Alert
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
            cv2.putText(frame, f"ALERT: {self.registered_person_name} (EYES CLOSED)", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        else:
            color = (255, 0, 0) # Blue for Tracking
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, self.registered_person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # Show the live data for tuning
            ear_text = f"EAR: {avg_ear:.2f} (Thresh: {self.calibrated_threshold:.2f})"
            count_text = f"Eye Closed Count: {self.eye_closed_counter}/{EYE_AR_CONSEC_FRAMES}"
            cv2.putText(frame, ear_text, (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(frame, count_text, (x, y + h + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        return frame, current_status


    def on_closing(self):
        """
        Handles the window close event.
        """
        if messagebox.askokcancel("Quit", "Do you want to exit the application?"):
            self.stop_video_stream()
            self.window.destroy()

# --- Main execution ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
!pip install beepy

In [ ]:
# !pip install playsound

In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os
import mediapipe as mp  # --- Import MediaPipe ---
import beepy as beep   # --- NEW: For sound alerts ---

# --- Configuration Constants ---

# --- New Dynamic Calibration Constants ---
# We no longer use a fixed EAR threshold.
# Instead, we set a personal threshold as a percentage of the user's open-eye value.
# e.g., if open EAR is 0.30, threshold will be 0.30 * 0.75 = 0.225
THRESHOLD_CALIBRATION_FACTOR = 0.75

# Sanity check: if the user's "open" EAR is below this,
# they probably tried to register with their eyes closed.
MIN_OPEN_EYE_EAR = 0.20

# How many consecutive frames of "closed" eyes trigger an alarm?
# User requested 10 seconds. Assuming ~20fps, 10 * 20 = 200 frames.
EYE_AR_CONSEC_FRAMES = 200

# --- NEW: Tracking "Grace Period" ---
# How many frames to keep searching for a person before declaring them "lost".
LOST_GRACE_FRAMES = 50 # ~2.5 seconds at 20fps

# Tracking constant: max distance a face can "jump" between frames.
MAX_TRACKING_JUMP_PX = 150

class SleepingAlertApp:
    def __init__(self, window):
        """
        Initialize the application.
        """
        self.window = window
        self.window.title("Sleeping Alert System (Calibrated + Sound)")
        self.window.geometry("800x700")

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- MediaPipe Face Mesh Initialization ---
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=5,  # Detect up to 5 faces
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # --- Logic & Registration State ---
        self.eye_closed_counter = 0
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None
        self.current_frame_for_registration = None
        self.registration_lock = threading.Lock()

        # --- New Calibration State Variables ---
        self.calibrated_open_ear = 0.0
        self.calibrated_threshold = 0.0
        
        # --- NEW: State variables for improved tracking and sound ---
        self.lost_tracker_counter = 0   # For grace period
        self.alert_sound_playing = False # To prevent continuous beeping

        # --- GUI Elements ---
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # --- GUI Layout Fix: Anchor controls to BOTTOM ---
        
        self.button_frame = tk.Frame(self.main_frame)
        self.button_frame.pack(side=tk.BOTTOM, pady=10)

        self.status_text = tk.StringVar()
        self.status_text.set("Ready. Press 'Start Camera' to begin.")
        self.status_label = tk.Label(self.main_frame, textvariable=self.status_text, font=("Arial", 14))
        self.status_label.pack(side=tk.BOTTOM, pady=5)

        # Video label expands to fill the remaining space
        self.video_label = tk.Label(self.main_frame, bg="black")
        self.video_label.pack(fill=tk.BOTH, expand=True)

        # --- Buttons ---
        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, font=("Arial", 12), width=15)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, font=("Arial", 12), width=15)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        self.register_button = tk.Button(self.button_frame, text="Register & Monitor", command=self.register_person, font=("Arial", 12), width=18)
        self.register_button.pack(side=tk.LEFT, padx=5)

        self.clear_button = tk.Button(self.button_frame, text="Clear Registration", command=self.clear_registration, font=("Arial", 12), width=18)
        self.clear_button.pack(side=tk.LEFT, padx=5)

        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        """
        Starts the video capture in a new thread.
        """
        if self.is_running:
            return

        try:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                raise IOError("Cannot open webcam.")
            
            self.is_running = True
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            self.status_text.set("Camera running. Click 'Register & Monitor'.")
        
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            if self.cap:
                self.cap.release()

    def stop_video_stream(self):
        """
        Signals the video loop to stop and resets all states.
        """
        if not self.is_running:
            return
            
        self.is_running = False
        if self.video_thread:
            self.video_thread.join(timeout=0.5) 
            
        if self.cap:
            self.cap.release()

        self.video_label.config(image=None)
        self.status_text.set("Camera stopped.")
        
        # Reset all logic states
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None
        self.eye_closed_counter = 0
        self.calibrated_open_ear = 0.0
        self.calibrated_threshold = 0.0
        self.lost_tracker_counter = 0
        self.alert_sound_playing = False

    def video_loop(self):
        """
        The main loop for video processing. Runs in a separate thread.
        """
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    self.status_text.set("Error: Can't read from camera.")
                    time.sleep(0.5)
                    continue

                frame = cv2.flip(frame, 1)
                
                with self.registration_lock:
                    self.current_frame_for_registration = frame.copy()
                
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Process the frame and get status
                processed_frame, status = self.process_frame_logic(frame, rgb_frame)

                self.status_text.set(status)

                # Convert for Tkinter display
                cv_img = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                
                w, h = self.video_label.winfo_width(), self.video_label.winfo_height()
                if w > 1 and h > 1:
                     pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)

                imgtk = ImageTk.PhotoImage(image=pil_img)
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

            except Exception as e:
                print(f"Error in video loop: {e}")
                self.is_running = False
            
            time.sleep(0.01)

        print("Video loop stopped.")

    def get_ear(self, eye_points, w, h):
        """
        Calculates the Eye Aspect Ratio (EAR) given 6 landmark points.
        """
        try:
            # Convert landmarks to (x, y) tuples
            p1 = (int(eye_points[0].x * w), int(eye_points[0].y * h))
            p4 = (int(eye_points[1].x * w), int(eye_points[1].y * h))
            p2 = (int(eye_points[2].x * w), int(eye_points[2].y * h))
            p6 = (int(eye_points[3].x * w), int(eye_points[3].y * h))
            p3 = (int(eye_points[4].x * w), int(eye_points[4].y * h))
            p5 = (int(eye_points[5].x * w), int(eye_points[5].y * h))

            def get_dist(p_a, p_b):
                return np.linalg.norm(np.array(p_a) - np.array(p_b))

            v_dist_1 = get_dist(p2, p6)
            v_dist_2 = get_dist(p3, p5)
            h_dist = get_dist(p1, p4)

            if h_dist == 0:
                return 0.3 # Default "open"

            ear = (v_dist_1 + v_dist_2) / (2.0 * h_dist)
            return ear
        except Exception:
            return 0.3 # Default "open"

    def get_faces_from_results(self, frame, results):
        """
        Extracts face data (box, center, EAR) from MediaPipe results.
        """
        faces_data = []
        h, w, _ = frame.shape

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # 1. Get Bounding Box
                x_min, y_min, x_max, y_max = w, h, 0, 0
                for landmark in face_landmarks.landmark:
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    if x < x_min: x_min = x
                    if y < y_min: y_min = y
                    if x > x_max: x_max = x
                    if y > y_max: y_max = y
                
                padding = 10
                x_min = max(0, x_min - padding)
                y_min = max(0, y_min - padding)
                x_max = min(w, x_max + padding)
                y_max = min(h, y_max + padding)
                box = (x_min, y_min, x_max - x_min, y_max - y_min)

                # 2. Get Nose Tip (Landmark 1) for tracking
                nose_tip = face_landmarks.landmark[1]
                center = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 3. Calculate Eye Aspect Ratio (EAR)
                RIGHT_EYE_EAR_POINTS_INDICES = [33, 133, 159, 145, 158, 153]
                LEFT_EYE_EAR_POINTS_INDICES = [362, 263, 386, 374, 385, 380]

                landmarks = face_landmarks.landmark
                right_eye_points = [landmarks[i] for i in RIGHT_EYE_EAR_POINTS_INDICES]
                left_eye_points = [landmarks[i] for i in LEFT_EYE_EAR_POINTS_INDICES]

                right_ear = self.get_ear(right_eye_points, w, h)
                left_ear = self.get_ear(left_eye_points, w, h)
                avg_ear = (left_ear + right_ear) / 2.0
                
                faces_data.append({'box': box, 'center': center, 'avg_ear': avg_ear})
        
        return faces_data

    def register_person(self):
        """
        Registers a person and dynamically calibrates the EAR threshold.
        """
        with self.registration_lock:
            if self.current_frame_for_registration is None:
                messagebox.showwarning("Registration Error", "Camera not ready. Please try again.")
                return
            
            rgb_frame_reg = cv2.cvtColor(self.current_frame_for_registration, cv2.COLOR_BGR2RGB)
            results = self.face_mesh.process(rgb_frame_reg)
            faces = self.get_faces_from_results(self.current_frame_for_registration, results)

        if len(faces) == 0:
            messagebox.showwarning("Registration Error", "No person detected. Please face the camera and try again.")
            return

        faces_by_area = sorted(faces, key=lambda f: f['box'][2] * f['box'][3], reverse=True)
        largest_face = faces_by_area[0]

        # --- DYNAMIC CALIBRATION ---
        current_ear = largest_face['avg_ear']
        
        if current_ear < MIN_OPEN_EYE_EAR:
            messagebox.showwarning("Registration Error", f"Eyes seem to be closed (EAR: {current_ear:.2f}).\nPlease open your eyes and try again.")
            return

        self.calibrated_open_ear = current_ear
        self.calibrated_threshold = current_ear * THRESHOLD_CALIBRATION_FACTOR
        
        name = simpledialog.askstring("Register Person", "Enter the person's name:", parent=self.window)
        if not name:
            messagebox.showwarning("Registration Cancelled", "Registration was cancelled.")
            return
        
        # --- Lock in the registration ---
        (x, y, w, h) = largest_face['box']
        self.is_monitoring = True
        self.registered_person_name = name
        self.registered_face_center_guess = largest_face['center']
        self.eye_closed_counter = 0
        self.lost_tracker_counter = 0 # Reset grace period counter

        status = f"Calibrated for {name}. Open EAR: {self.calibrated_open_ear:.2f}, Threshold: {self.calibrated_threshold:.2f}"
        self.status_text.set(status)
        messagebox.showinfo("Registration Complete", status)

    def clear_registration(self):
        """
        Clears the current registration and stops monitoring.
        """
        if not self.is_monitoring:
            messagebox.showwarning("Info", "No person is currently registered.")
            return
        
        # Reset all logic states
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None
        self.eye_closed_counter = 0
        self.calibrated_open_ear = 0.0
        self.calibrated_threshold = 0.0
        self.lost_tracker_counter = 0
        self.alert_sound_playing = False

        self.status_text.set("Monitoring stopped. Ready to register a new person.")


    def process_frame_logic(self, frame, rgb_frame):
        """
        Main logic: finds faces, tracks registered person, checks EAR.
        """
        results = self.face_mesh.process(rgb_frame)
        faces = self.get_faces_from_results(frame, results)

        # If not monitoring, just show all faces
        if not self.is_monitoring:
            current_status = "Click 'Register & Monitor' to begin."
            for face_data in faces:
                (x, y, w, h) = face_data['box']
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2)
            return frame, current_status

        # --- If we ARE monitoring ---
        
        # --- NEW TRACKING LOGIC ---
        tracked_face_data = None
        current_center = None

        if len(faces) > 0:
            # Find the face closest to our last known center
            min_dist = float('inf')
            for face_data in faces:
                center = face_data['center']
                dist = np.linalg.norm(np.array(center) - np.array(self.registered_face_center_guess))
                if dist < min_dist:
                    min_dist = dist
                    tracked_face_data = face_data
                    current_center = center
            
            if min_dist > MAX_TRACKING_JUMP_PX:
                # The closest face is too far, it's probably someone else
                tracked_face_data = None
        
        if tracked_face_data is None:
            # --- We did NOT find our person this frame ---
            self.lost_tracker_counter += 1
            
            if self.lost_tracker_counter > LOST_GRACE_FRAMES:
                # We've lost them for too long
                current_status = f"MONITORING: {self.registered_person_name} lost!"
                self.eye_closed_counter = 0
                self.alert_sound_playing = False
            else:
                # We are in the grace period, keep searching
                current_status = f"MONITORING: Searching for {self.registered_person_name}..."
            
            return frame, current_status
        
        # --- We DID find our person (tracked_face_data is not None) ---
        self.lost_tracker_counter = 0 # We found them, reset grace period
        self.registered_face_center_guess = current_center
        (x, y, w, h) = tracked_face_data['box']
        avg_ear = tracked_face_data['avg_ear']
        alert_triggered = False
        
        # --- CALIBRATED Eye Aspect Ratio (EAR) Logic ---
        if avg_ear < self.calibrated_threshold:
            self.eye_closed_counter += 1
        else:
            self.eye_closed_counter = 0
            self.alert_sound_playing = False # Reset sound flag

            # --- NEW: Continuous Re-calibration ---
            # Slowly adjust the baseline "open" EAR value to account for
            # changes in lighting, head angle, glasses, etc.
            alpha = 0.01 # Small learning rate
            self.calibrated_open_ear = (self.calibrated_open_ear * (1 - alpha)) + (avg_ear * alpha)
            self.calibrated_threshold = self.calibrated_open_ear * THRESHOLD_CALIBRATION_FACTOR
        
        if self.eye_closed_counter > EYE_AR_CONSEC_FRAMES:
            current_status = f"!!! ALERT: {self.registered_person_name} EYES CLOSED !!!"
            alert_triggered = True
        else:
            # Don't override the status if we're alerting
            current_status = f"Monitoring: {self.registered_person_name} (Eyes Open)"
        
        # --- Draw on the frame ---
        if alert_triggered:
            color = (0, 0, 255) # Red for Alert
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
            cv2.putText(frame, f"ALERT: {self.registered_person_name} (EYES CLOSED)", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # --- NEW: Play Sound Alert ---
            if not self.alert_sound_playing:
                # Start playing the sound in a non-blocking thread
                # This prevents the video from freezing
                threading.Thread(target=lambda: beep.beep(sound=3), daemon=True).start()
                self.alert_sound_playing = True # Set flag so it only beeps once
        else:
            color = (255, 0, 0) # Blue for Tracking
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, self.registered_person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # Show the live data for tuning
            ear_text = f"EAR: {avg_ear:.2f} (Thresh: {self.calibrated_threshold:.2f})"
            count_text = f"Eye Closed Count: {self.eye_closed_counter}/{EYE_AR_CONSEC_FRAMES}"
            cv2.putText(frame, ear_text, (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(frame, count_text, (x, y + h + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        return frame, current_status


    def on_closing(self):
        """
        Handles the window close event.
        """
        if messagebox.askokcancel("Quit", "Do you want to exit the application?"):
            self.stop_video_stream()
            self.window.destroy()

# --- Main execution ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except ImportError:
        print("\n--- ERROR ---")
        print("Could not import 'beepy'.")
        print("Please install it by running: pip install beepy")
        print("---------------")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os
import mediapipe as mp  # --- Import MediaPipe ---
from playsound import playsound # --- NEW: Replaced beepy with playsound ---

# --- Configuration Constants ---

# --- NEW: Path to the alert sound ---
# NOTE: Change this to the full path of your .wav file
# e.g., "D:\\GIT_HUB\\12_Final_Projects_of_all\\03_deep_learning\\beep-01a.wav"
# Make sure to use double backslashes (\\) on Windows!
ALERT_SOUND_FILE_PATH = "D:\\GIT_HUB\\12_Final_Projects_of_all\\03_deep_learning\\beep-01a.wav"


# --- New Dynamic Calibration Constants ---
# We no longer use a fixed EAR threshold.
# Instead, we set a personal threshold as a percentage of the user's open-eye value.
# e.g., if open EAR is 0.30, threshold will be 0.30 * 0.75 = 0.225
THRESHOLD_CALIBRATION_FACTOR = 0.75

# Sanity check: if the user's "open" EAR is below this,
# they probably tried to register with their eyes closed.
MIN_OPEN_EYE_EAR = 0.20

# How many consecutive frames of "closed" eyes trigger an alarm?
# User requested 10 seconds. Assuming ~20fps, 10 * 20 = 200 frames.
EYE_AR_CONSEC_FRAMES = 200

# --- NEW: Tracking "Grace Period" ---
# How many frames to keep searching for a person before declaring them "lost".
LOST_GRACE_FRAMES = 50 # ~2.5 seconds at 20fps

# Tracking constant: max distance a face can "jump" between frames.
MAX_TRACKING_JUMP_PX = 150

class SleepingAlertApp:
    def __init__(self, window):
        """
        Initialize the application.
        """
        self.window = window
        self.window.title("Sleeping Alert System (Calibrated + Sound File)")
        self.window.geometry("800x700")

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- MediaPipe Face Mesh Initialization ---
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=5,  # Detect up to 5 faces
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # --- Logic & Registration State ---
        self.eye_closed_counter = 0
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None
        self.current_frame_for_registration = None
        self.registration_lock = threading.Lock()

        # --- New Calibration State Variables ---
        self.calibrated_open_ear = 0.0
        self.calibrated_threshold = 0.0
        
        # --- NEW: State variables for improved tracking and sound ---
        self.lost_tracker_counter = 0   # For grace period
        self.alert_sound_playing = False # To prevent continuous beeping

        # --- GUI Elements ---
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # --- GUI Layout Fix: Anchor controls to BOTTOM ---
        
        self.button_frame = tk.Frame(self.main_frame)
        self.button_frame.pack(side=tk.BOTTOM, pady=10)

        self.status_text = tk.StringVar()
        self.status_text.set("Ready. Press 'Start Camera' to begin.")
        self.status_label = tk.Label(self.main_frame, textvariable=self.status_text, font=("Arial", 14))
        self.status_label.pack(side=tk.BOTTOM, pady=5)

        # Video label expands to fill the remaining space
        self.video_label = tk.Label(self.main_frame, bg="black")
        self.video_label.pack(fill=tk.BOTH, expand=True)

        # --- Buttons ---
        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, font=("Arial", 12), width=15)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, font=("Arial", 12), width=15)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        self.register_button = tk.Button(self.button_frame, text="Register & Monitor", command=self.register_person, font=("Arial", 12), width=18)
        self.register_button.pack(side=tk.LEFT, padx=5)

        self.clear_button = tk.Button(self.button_frame, text="Clear Registration", command=self.clear_registration, font=("Arial", 12), width=18)
        self.clear_button.pack(side=tk.LEFT, padx=5)

        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        """
        Starts the video capture in a new thread.
        """
        if self.is_running:
            return

        try:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                raise IOError("Cannot open webcam.")
            
            self.is_running = True
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            self.status_text.set("Camera running. Click 'Register & Monitor'.")
        
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            if self.cap:
                self.cap.release()

    def stop_video_stream(self):
        """
        Signals the video loop to stop and resets all states.
        """
        if not self.is_running:
            return
            
        self.is_running = False
        if self.video_thread:
            self.video_thread.join(timeout=0.5) 
            
        if self.cap:
            self.cap.release()

        self.video_label.config(image=None)
        self.status_text.set("Camera stopped.")
        
        # Reset all logic states
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None
        self.eye_closed_counter = 0
        self.calibrated_open_ear = 0.0
        self.calibrated_threshold = 0.0
        self.lost_tracker_counter = 0
        self.alert_sound_playing = False

    def video_loop(self):
        """
        The main loop for video processing. Runs in a separate thread.
        """
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    self.status_text.set("Error: Can't read from camera.")
                    time.sleep(0.5)
                    continue

                frame = cv2.flip(frame, 1)
                
                with self.registration_lock:
                    self.current_frame_for_registration = frame.copy()
                
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                # Process the frame and get status
                processed_frame, status = self.process_frame_logic(frame, rgb_frame)

                self.status_text.set(status)

                # Convert for Tkinter display
                cv_img = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                
                w, h = self.video_label.winfo_width(), self.video_label.winfo_height()
                if w > 1 and h > 1:
                     pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)

                imgtk = ImageTk.PhotoImage(image=pil_img)
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

            except Exception as e:
                print(f"Error in video loop: {e}")
                self.is_running = False
            
            time.sleep(0.01)

        print("Video loop stopped.")

    def get_ear(self, eye_points, w, h):
        """
        Calculates the Eye Aspect Ratio (EAR) given 6 landmark points.
        """
        try:
            # Convert landmarks to (x, y) tuples
            p1 = (int(eye_points[0].x * w), int(eye_points[0].y * h))
            p4 = (int(eye_points[1].x * w), int(eye_points[1].y * h))
            p2 = (int(eye_points[2].x * w), int(eye_points[2].y * h))
            p6 = (int(eye_points[3].x * w), int(eye_points[3].y * h))
            p3 = (int(eye_points[4].x * w), int(eye_points[4].y * h))
            p5 = (int(eye_points[5].x * w), int(eye_points[5].y * h))

            def get_dist(p_a, p_b):
                return np.linalg.norm(np.array(p_a) - np.array(p_b))

            v_dist_1 = get_dist(p2, p6)
            v_dist_2 = get_dist(p3, p5)
            h_dist = get_dist(p1, p4)

            if h_dist == 0:
                return 0.3 # Default "open"

            ear = (v_dist_1 + v_dist_2) / (2.0 * h_dist)
            return ear
        except Exception:
            return 0.3 # Default "open"

    def get_faces_from_results(self, frame, results):
        """
        Extracts face data (box, center, EAR) from MediaPipe results.
        """
        faces_data = []
        h, w, _ = frame.shape

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # 1. Get Bounding Box
                x_min, y_min, x_max, y_max = w, h, 0, 0
                for landmark in face_landmarks.landmark:
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    if x < x_min: x_min = x
                    if y < y_min: y_min = y
                    if x > x_max: x_max = x
                    if y > y_max: y_max = y
                
                padding = 10
                x_min = max(0, x_min - padding)
                y_min = max(0, y_min - padding)
                x_max = min(w, x_max + padding)
                y_max = min(h, y_max + padding)
                box = (x_min, y_min, x_max - x_min, y_max - y_min)

                # 2. Get Nose Tip (Landmark 1) for tracking
                nose_tip = face_landmarks.landmark[1]
                center = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 3. Calculate Eye Aspect Ratio (EAR)
                RIGHT_EYE_EAR_POINTS_INDICES = [33, 133, 159, 145, 158, 153]
                LEFT_EYE_EAR_POINTS_INDICES = [362, 263, 386, 374, 385, 380]

                landmarks = face_landmarks.landmark
                right_eye_points = [landmarks[i] for i in RIGHT_EYE_EAR_POINTS_INDICES]
                left_eye_points = [landmarks[i] for i in LEFT_EYE_EAR_POINTS_INDICES]

                right_ear = self.get_ear(right_eye_points, w, h)
                left_ear = self.get_ear(left_eye_points, w, h)
                avg_ear = (left_ear + right_ear) / 2.0
                
                faces_data.append({'box': box, 'center': center, 'avg_ear': avg_ear})
        
        return faces_data

    def register_person(self):
        """
        Registers a person and dynamically calibrates the EAR threshold.
        """
        with self.registration_lock:
            if self.current_frame_for_registration is None:
                messagebox.showwarning("Registration Error", "Camera not ready. Please try again.")
                return
            
            rgb_frame_reg = cv2.cvtColor(self.current_frame_for_registration, cv2.COLOR_BGR2RGB)
            results = self.face_mesh.process(rgb_frame_reg)
            faces = self.get_faces_from_results(self.current_frame_for_registration, results)

        if len(faces) == 0:
            messagebox.showwarning("Registration Error", "No person detected. Please face the camera and try again.")
            return

        faces_by_area = sorted(faces, key=lambda f: f['box'][2] * f['box'][3], reverse=True)
        largest_face = faces_by_area[0]

        # --- DYNAMIC CALIBRATION ---
        current_ear = largest_face['avg_ear']
        
        if current_ear < MIN_OPEN_EYE_EAR:
            messagebox.showwarning("Registration Error", f"Eyes seem to be closed (EAR: {current_ear:.2f}).\nPlease open your eyes and try again.")
            return

        self.calibrated_open_ear = current_ear
        self.calibrated_threshold = current_ear * THRESHOLD_CALIBRATION_FACTOR
        
        name = simpledialog.askstring("Register Person", "Enter the person's name:", parent=self.window)
        if not name:
            messagebox.showwarning("Registration Cancelled", "Registration was cancelled.")
            return
        
        # --- Lock in the registration ---
        (x, y, w, h) = largest_face['box']
        self.is_monitoring = True
        self.registered_person_name = name
        self.registered_face_center_guess = largest_face['center']
        self.eye_closed_counter = 0
        self.lost_tracker_counter = 0 # Reset grace period counter

        status = f"Calibrated for {name}. Open EAR: {self.calibrated_open_ear:.2f}, Threshold: {self.calibrated_threshold:.2f}"
        self.status_text.set(status)
        messagebox.showinfo("Registration Complete", status)

    def clear_registration(self):
        """
        Clears the current registration and stops monitoring.
        """
        if not self.is_monitoring:
            messagebox.showwarning("Info", "No person is currently registered.")
            return
        
        # Reset all logic states
        self.is_monitoring = False
        self.registered_face_center_guess = None
        self.registered_person_name = None
        self.eye_closed_counter = 0
        self.calibrated_open_ear = 0.0
        self.calibrated_threshold = 0.0
        self.lost_tracker_counter = 0
        self.alert_sound_playing = False

        self.status_text.set("Monitoring stopped. Ready to register a new person.")

    # --- NEW: Helper function to play sound in a thread ---
    def play_alert_sound(self):
        """
        Plays the alert sound file.
        Includes a try-except block in case the file is missing or invalid.
        """
        if not os.path.exists(ALERT_SOUND_FILE_PATH):
            print(f"Alert Sound Error: File not found at {ALERT_SOUND_FILE_PATH}")
            print("!!! ALERT (SOUND FILE MISSING) !!!")
            return

        try:
            playsound(ALERT_SOUND_FILE_PATH)
        except Exception as e:
            print(f"Error playing sound file {ALERT_SOUND_FILE_PATH}: {e}")
            # Fallback to a simple print if sound fails
            print("!!! ALERT BEEP !!!")

    def process_frame_logic(self, frame, rgb_frame):
        """
        Main logic: finds faces, tracks registered person, checks EAR.
        """
        results = self.face_mesh.process(rgb_frame)
        faces = self.get_faces_from_results(frame, results)

        # If not monitoring, just show all faces
        if not self.is_monitoring:
            current_status = "Click 'Register & Monitor' to begin."
            for face_data in faces:
                (x, y, w, h) = face_data['box']
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2)
            return frame, current_status

        # --- If we ARE monitoring ---
        
        # --- NEW TRACKING LOGIC ---
        tracked_face_data = None
        current_center = None

        if len(faces) > 0:
            # Find the face closest to our last known center
            min_dist = float('inf')
            for face_data in faces:
                center = face_data['center']
                dist = np.linalg.norm(np.array(center) - np.array(self.registered_face_center_guess))
                if dist < min_dist:
                    min_dist = dist
                    tracked_face_data = face_data
                    current_center = center
            
            if min_dist > MAX_TRACKING_JUMP_PX:
                # The closest face is too far, it's probably someone else
                tracked_face_data = None
        
        if tracked_face_data is None:
            # --- We did NOT find our person this frame ---
            self.lost_tracker_counter += 1
            
            if self.lost_tracker_counter > LOST_GRACE_FRAMES:
                # We've lost them for too long
                current_status = f"MONITORING: {self.registered_person_name} lost!"
                self.eye_closed_counter = 0
                self.alert_sound_playing = False
            else:
                # We are in the grace period, keep searching
                current_status = f"MONITORING: Searching for {self.registered_person_name}..."
            
            return frame, current_status
        
        # --- We DID find our person (tracked_face_data is not None) ---
        self.lost_tracker_counter = 0 # We found them, reset grace period
        self.registered_face_center_guess = current_center
        (x, y, w, h) = tracked_face_data['box']
        avg_ear = tracked_face_data['avg_ear']
        alert_triggered = False
        
        # --- CALIBRATED Eye Aspect Ratio (EAR) Logic ---
        if avg_ear < self.calibrated_threshold:
            self.eye_closed_counter += 1
        else:
            self.eye_closed_counter = 0
            self.alert_sound_playing = False # Reset sound flag

            # --- NEW: Continuous Re-calibration ---
            # Slowly adjust the baseline "open" EAR value to account for
            # changes in lighting, head angle, glasses, etc.
            alpha = 0.01 # Small learning rate
            self.calibrated_open_ear = (self.calibrated_open_ear * (1 - alpha)) + (avg_ear * alpha)
            self.calibrated_threshold = self.calibrated_open_ear * THRESHOLD_CALIBRATION_FACTOR
        
        if self.eye_closed_counter > EYE_AR_CONSEC_FRAMES:
            current_status = f"!!! ALERT: {self.registered_person_name} EYES CLOSED !!!"
            alert_triggered = True
        else:
            # Don't override the status if we're alerting
            current_status = f"Monitoring: {self.registered_person_name} (Eyes Open)"
        
        # --- Draw on the frame ---
        if alert_triggered:
            color = (0, 0, 255) # Red for Alert
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
            cv2.putText(frame, f"ALERT: {self.registered_person_name} (EYES CLOSED)", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # --- MODIFIED: Play Sound Alert ---
            if not self.alert_sound_playing:
                # Start playing the sound in a non-blocking thread
                # This prevents the video from freezing
                threading.Thread(target=self.play_alert_sound, daemon=True).start()
                self.alert_sound_playing = True # Set flag so it only beeps once
        else:
            color = (255, 0, 0) # Blue for Tracking
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, self.registered_person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # Show the live data for tuning
            ear_text = f"EAR: {avg_ear:.2f} (Thresh: {self.calibrated_threshold:.2f})"
            count_text = f"Eye Closed Count: {self.eye_closed_counter}/{EYE_AR_CONSEC_FRAMES}"
            cv2.putText(frame, ear_text, (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(frame, count_text, (x, y + h + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        return frame, current_status


    def on_closing(self):
        """
        Handles the window close event.
        """
        if messagebox.askokcancel("Quit", "Do you want to exit the application?"):
            self.stop_video_stream()
            self.window.destroy()

# --- Main execution ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except ImportError:
        print("\n--- ERROR ---")
        print("Could not import 'playsound'.")
        print("Please install it by running: pip install playsound")
        print("---------------")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os
import mediapipe as mp  # --- Import MediaPipe ---
import winsound        # --- NEW: Replaced playsound with winsound (built-in on Windows) ---

# --- Configuration Constants ---

# --- Path to the alert sound ---
# Make sure to use double backslashes (\\) on Windows!
ALERT_SOUND_FILE_PATH = "D:\\GIT_HUB\\12_Final_Projects_of_all\\03_deep_learning\\beep-01a.wav"

# --- Dynamic Calibration Constants ---
THRESHOLD_CALIBRATION_FACTOR = 0.75
MIN_OPEN_EYE_EAR = 0.20
EYE_AR_CONSEC_FRAMES = 200 # ~10 seconds at 20fps

# --- Tracking Constants ---
LOST_GRACE_FRAMES = 50 # ~2.5 seconds
MAX_TRACKING_JUMP_PX = 150 # Max distance a face can move between frames

class SleepingAlertApp:
    def __init__(self, window):
        """
        Initialize the application.
        """
        self.window = window
        self.window.title("Multi-Person Sleeping Alert System")
        self.window.geometry("1000x700") # Made window wider for sidebar

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- MediaPipe Face Mesh Initialization ---
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=10,  # Detect more faces
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # --- NEW: Multi-Person Logic & Registration State ---
        # Replaces all single-person variables
        self.registered_people = {} # Main dictionary to hold all registered people
        self.registration_lock = threading.Lock()
        self.current_frame_for_registration = None

        # --- GUI Elements ---
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # --- Bottom Frame (Buttons & Status) ---
        self.bottom_frame = tk.Frame(self.main_frame)
        self.bottom_frame.pack(side=tk.BOTTOM, fill=tk.X, pady=5)
        
        self.status_text = tk.StringVar()
        self.status_text.set("Ready. Press 'Start Camera' to begin.")
        self.status_label = tk.Label(self.bottom_frame, textvariable=self.status_text, font=("Arial", 14))
        self.status_label.pack(side=tk.LEFT, padx=10)
        
        self.button_frame = tk.Frame(self.bottom_frame)
        self.button_frame.pack(side=tk.RIGHT)

        # --- Content Frame (Video & Sidebar) ---
        self.content_frame = tk.Frame(self.main_frame)
        self.content_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # Video label expands to fill the remaining space
        self.video_label = tk.Label(self.content_frame, bg="black")
        self.video_label.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # --- NEW: Sidebar for Registered People List ---
        self.sidebar_frame = tk.Frame(self.content_frame, width=250, bg="#f0f0f0", relief=tk.SUNKEN, borderwidth=2)
        self.sidebar_frame.pack(side=tk.RIGHT, fill=tk.Y)
        self.sidebar_frame.pack_propagate(False) # Prevent frame from shrinking

        self.sidebar_title = tk.Label(self.sidebar_frame, text="Registered People", font=("Arial", 16, "bold"), bg="#f0f0f0")
        self.sidebar_title.pack(pady=10, padx=10, anchor="w")

        self.registered_list_var = tk.StringVar()
        self.registered_list_display = tk.Label(self.sidebar_frame, textvariable=self.registered_list_var, font=("Arial", 12), bg="#f0f0f0", justify=tk.LEFT, anchor="nw")
        self.registered_list_display.pack(pady=5, padx=10, fill=tk.X, anchor="nw")
        self.registered_list_var.set("None")

        # --- Buttons (in the bottom_frame) ---
        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, font=("Arial", 12), width=15)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, font=("Arial", 12), width=15)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        self.register_button = tk.Button(self.button_frame, text="Register & Monitor", command=self.register_person, font=("Arial", 12), width=18)
        self.register_button.pack(side=tk.LEFT, padx=5)

        # Renamed "Clear Registration" to "Clear All"
        self.clear_button = tk.Button(self.button_frame, text="Clear All", command=self.clear_all_registrations, font=("Arial", 12), width=18)
        self.clear_button.pack(side=tk.LEFT, padx=5)

        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        if self.is_running:
            return
        try:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                raise IOError("Cannot open webcam.")
            self.is_running = True
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            self.status_text.set("Camera running. Click 'Register & Monitor'.")
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            if self.cap: self.cap.release()

    def stop_video_stream(self):
        if not self.is_running:
            return
        self.is_running = False
        if self.video_thread:
            self.video_thread.join(timeout=0.5) 
        if self.cap:
            self.cap.release()
        self.video_label.config(image=None)
        self.status_text.set("Camera stopped.")
        self.clear_all_registrations() # Clear people when camera stops

    def video_loop(self):
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    self.status_text.set("Error: Can't read from camera.")
                    time.sleep(0.5)
                    continue

                frame = cv2.flip(frame, 1)
                with self.registration_lock:
                    self.current_frame_for_registration = frame.copy()
                
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                processed_frame, status, list_text = self.process_frame_logic(frame, rgb_frame)
                self.status_text.set(status)
                self.registered_list_var.set(list_text if list_text else "None")

                cv_img = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                w, h = self.video_label.winfo_width(), self.video_label.winfo_height()
                if w > 1 and h > 1:
                     pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)
                imgtk = ImageTk.PhotoImage(image=pil_img)
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)
            except Exception as e:
                print(f"Error in video loop: {e}")
                self.is_running = False
            time.sleep(0.01)
        print("Video loop stopped.")

    def get_ear(self, eye_points, w, h):
        try:
            p1 = (int(eye_points[0].x * w), int(eye_points[0].y * h))
            p4 = (int(eye_points[1].x * w), int(eye_points[1].y * h))
            p2 = (int(eye_points[2].x * w), int(eye_points[2].y * h))
            p6 = (int(eye_points[3].x * w), int(eye_points[3].y * h))
            p3 = (int(eye_points[4].x * w), int(eye_points[4].y * h))
            p5 = (int(eye_points[5].x * w), int(eye_points[5].y * h))
            def get_dist(p_a, p_b):
                return np.linalg.norm(np.array(p_a) - np.array(p_b))
            v_dist_1 = get_dist(p2, p6)
            v_dist_2 = get_dist(p3, p5)
            h_dist = get_dist(p1, p4)
            if h_dist == 0: return 0.3
            ear = (v_dist_1 + v_dist_2) / (2.0 * h_dist)
            return ear
        except Exception:
            return 0.3

    def get_faces_from_results(self, frame, results):
        faces_data = []
        h, w, _ = frame.shape
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                x_min, y_min, x_max, y_max = w, h, 0, 0
                for landmark in face_landmarks.landmark:
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    if x < x_min: x_min = x
                    if y < y_min: y_min = y
                    if x > x_max: x_max = x
                    if y > y_max: y_max = y
                padding = 10
                x_min = max(0, x_min - padding)
                y_min = max(0, y_min - padding)
                x_max = min(w, x_max + padding)
                y_max = min(h, y_max + padding)
                box = (x_min, y_min, x_max - x_min, y_max - y_min)
                nose_tip = face_landmarks.landmark[1]
                center = (int(nose_tip.x * w), int(nose_tip.y * h))
                RIGHT_EYE_EAR_POINTS_INDICES = [33, 133, 159, 145, 158, 153]
                LEFT_EYE_EAR_POINTS_INDICES = [362, 263, 386, 374, 385, 380]
                landmarks = face_landmarks.landmark
                right_eye_points = [landmarks[i] for i in RIGHT_EYE_EAR_POINTS_INDICES]
                left_eye_points = [landmarks[i] for i in LEFT_EYE_EAR_POINTS_INDICES]
                right_ear = self.get_ear(right_eye_points, w, h)
                left_ear = self.get_ear(left_eye_points, w, h)
                avg_ear = (left_ear + right_ear) / 2.0
                faces_data.append({'box': box, 'center': center, 'avg_ear': avg_ear})
        return faces_data

    def register_person(self):
        with self.registration_lock:
            if self.current_frame_for_registration is None:
                messagebox.showwarning("Registration Error", "Camera not ready. Please try again.")
                return
            rgb_frame_reg = cv2.cvtColor(self.current_frame_for_registration, cv2.COLOR_BGR2RGB)
            results = self.face_mesh.process(rgb_frame_reg)
            faces = self.get_faces_from_results(self.current_frame_for_registration, results)

        if len(faces) == 0:
            messagebox.showwarning("Registration Error", "No person detected. Please face the camera and try again.")
            return

        faces_by_area = sorted(faces, key=lambda f: f['box'][2] * f['box'][3], reverse=True)
        largest_face = faces_by_area[0]
        current_ear = largest_face['avg_ear']
        
        if current_ear < MIN_OPEN_EYE_EAR:
            messagebox.showwarning("Registration Error", f"Eyes seem to be closed (EAR: {current_ear:.2f}).\nPlease open your eyes and try again.")
            return

        calibrated_threshold = current_ear * THRESHOLD_CALIBRATION_FACTOR
        name = simpledialog.askstring("Register Person", "Enter a unique name:", parent=self.window)
        
        if not name:
            messagebox.showwarning("Registration Cancelled", "Registration was cancelled.")
            return
            
        if name in self.registered_people:
            messagebox.showwarning("Registration Error", f"The name '{name}' is already registered. Please use a unique name.")
            return
        
        # --- Add new person to the dictionary ---
        self.registered_people[name] = {
            'center': largest_face['center'],
            'open_ear': current_ear,
            'threshold': calibrated_threshold,
            'eye_counter': 0,
            'lost_counter': 0,
            'alert_sounding': False,
            'status': 'Calibrated' # For the sidebar list
        }

        status = f"Calibrated for {name}. Open EAR: {current_ear:.2f}, Threshold: {calibrated_threshold:.2f}"
        self.status_text.set(f"Registered {name}. Monitoring {len(self.registered_people)} person(s).")
        messagebox.showinfo("Registration Complete", status)

    def clear_all_registrations(self):
        if not self.registered_people:
            # No need to show a warning if it's already empty
            return
        
        self.registered_people = {}
        self.status_text.set("All registrations cleared. Ready to register.")
        self.registered_list_var.set("None")

    # --- Replaced play_alert_sound with winsound ---
    def play_alert_sound(self):
        if not os.path.exists(ALERT_SOUND_FILE_PATH):
            print(f"Alert Sound Error: File not found at {ALERT_SOUND_FILE_PATH}")
            print("!!! ALERT (SOUND FILE MISSING) !!!")
            return
        try:
            # Play sound asynchronously to not block the main thread
            winsound.PlaySound(ALERT_SOUND_FILE_PATH, winsound.SND_FILENAME | winsound.SND_ASYNC)
        except Exception as e:
            print(f"Error playing sound file {ALERT_SOUND_FILE_PATH}: {e}")
            print("!!! ALERT BEEP !!!")

    # --- NEW: Helper function to draw on face and check logic ---
    def process_person(self, frame, person_name, person_data, face_data):
        """
        Processes a single person, either found (face_data not None) or lost.
        Updates their state and draws on the frame.
        """
        alert_triggered = False
        
        if face_data is None:
            # --- Person is LOST ---
            person_data['lost_counter'] += 1
            if person_data['lost_counter'] > LOST_GRACE_FRAMES:
                person_data['status'] = 'Lost'
                person_data['eye_counter'] = 0
                person_data['alert_sounding'] = False
            else:
                person_data['status'] = 'Searching...'
            return # Stop processing this person
        
        # --- Person is FOUND ---
        person_data['lost_counter'] = 0
        person_data['center'] = face_data['center']
        avg_ear = face_data['avg_ear']
        
        if avg_ear < person_data['threshold']:
            person_data['eye_counter'] += 1
        else:
            person_data['eye_counter'] = 0
            person_data['alert_sounding'] = False
            # Continuous Re-calibration
            alpha = 0.01
            person_data['open_ear'] = (person_data['open_ear'] * (1 - alpha)) + (avg_ear * alpha)
            person_data['threshold'] = person_data['open_ear'] * THRESHOLD_CALIBRATION_FACTOR
        
        if person_data['eye_counter'] > EYE_AR_CONSEC_FRAMES:
            person_data['status'] = '!!! ALERT !!!'
            alert_triggered = True
        else:
            person_data['status'] = 'Tracking'
        
        # --- Draw on Frame ---
        (x, y, w, h) = face_data['box']
        
        if alert_triggered:
            color = (0, 0, 255) # Red
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
            cv2.putText(frame, f"ALERT: {person_name} (EYES CLOSED)", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            if not person_data['alert_sounding']:
                threading.Thread(target=self.play_alert_sound, daemon=True).start()
                person_data['alert_sounding'] = True
        else:
            color = (255, 0, 0) # Blue
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            ear_text = f"EAR: {avg_ear:.2f} (Th: {person_data['threshold']:.2f})"
            count_text = f"Count: {person_data['eye_counter']}/{EYE_AR_CONSEC_FRAMES}"
            cv2.putText(frame, ear_text, (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(frame, count_text, (x, y + h + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)


    def process_frame_logic(self, frame, rgb_frame):
        results = self.face_mesh.process(rgb_frame)
        faces = self.get_faces_from_results(frame, results)

        if not self.registered_people:
            # Not monitoring anyone, just draw all faces
            for face_data in faces:
                (x, y, w, h) = face_data['box']
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2)
            return frame, "Click 'Register & Monitor' to begin.", ""

        # --- Multi-Person Tracking Logic ---
        unmatched_face_indices = set(range(len(faces)))
        matches = {} # Stores {person_name: face_index}
        
        # --- Greedily assign best match for each person ---
        for name, person_data in self.registered_people.items():
            best_dist = float('inf')
            best_face_idx = -1
            for i in unmatched_face_indices: # Only check against unused faces
                dist = np.linalg.norm(np.array(faces[i]['center']) - np.array(person_data['center']))
                if dist < best_dist and dist < MAX_TRACKING_JUMP_PX:
                    best_dist = dist
                    best_face_idx = i
            
            if best_face_idx != -1:
                matches[name] = best_face_idx
                unmatched_face_indices.remove(best_face_idx) # This face is now taken

        list_display_text = []
        # --- Process all people (matched or lost) ---
        for name, person_data in self.registered_people.items():
            face_data = None
            if name in matches:
                face_data = faces[matches[name]] # Get the matched face data
            
            # This function updates the person's state and draws on the frame
            self.process_person(frame, name, person_data, face_data)
            
            list_display_text.append(f"{name}: {person_data['status']}")

        # --- Draw remaining unmatched (unregistered) faces ---
        for i in unmatched_face_indices:
            (x, y, w, h) = faces[i]['box']
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2)

        status = f"Monitoring {len(self.registered_people)} person(s)."
        return frame, status, "\n".join(list_display_text)

    def on_closing(self):
        if messagebox.askokcancel("Quit", "Do you want to exit the application?"):
            self.stop_video_stream()
            self.window.destroy()

# --- Main execution ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except ImportError:
        print("\n--- ERROR ---")
        print("Could not import one or more required libraries.")
        print("Please ensure you have the following installed:")
        print("pip install opencv-python pillow numpy mediapipe")
        print("---------------")
    except Exception as e:
        print(f"An error occurred: {e}")

In [4]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os
import mediapipe as mp  # --- Import MediaPipe ---
import winsound        # --- Using built-in winsound for Windows ---

# --- Configuration Constants ---

# --- Path to the alert sound ---
# Make sure to use double backslashes (\\) on Windows!
ALERT_SOUND_FILE_PATH = "D:\\GIT_HUB\\12_Final_Projects_of_all\\03_deep_learning\\beep-01a.wav"

# --- NEW: Sound loop configuration ---
# Time in seconds between playing the alert sound
ALERT_SOUND_INTERVAL = 2.0 # Play sound every 2 seconds

# --- Dynamic Calibration Constants ---
THRESHOLD_CALIBRATION_FACTOR = 0.75
MIN_OPEN_EYE_EAR = 0.20
EYE_AR_CONSEC_FRAMES = 200 # ~10 seconds at 20fps

# --- Tracking Constants ---
LOST_GRACE_FRAMES = 50 # ~2.5 seconds
MAX_TRACKING_JUMP_PX = 150 # Max distance a face can move between frames

class SleepingAlertApp:
    def __init__(self, window):
        """
        Initialize the application.
        """
        self.window = window
        self.window.title("Multi-Person Sleeping Alert System (v2.0)")
        self.window.geometry("1000x700") # Made window wider for sidebar
        self.window.minsize(800, 600) # Set a minimum size

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- MediaPipe Face Mesh Initialization ---
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=10,  # Detect more faces
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # --- Multi-Person Logic & Registration State ---
        self.registered_people = {} # Main dictionary to hold all registered people
        self.registration_lock = threading.Lock()
        self.current_frame_for_registration = None

        # --- GUI REBUILD: Using .grid() for stable layout ---
        
        # Configure the main window's grid
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        self.main_frame.rowconfigure(0, weight=1) # Content (video/sidebar) expands
        self.main_frame.rowconfigure(1, weight=0) # Bottom bar does not expand
        self.main_frame.columnconfigure(0, weight=1) # Main column expands
        
        # --- 1. Content Frame (Video & Sidebar) ---
        self.content_frame = tk.Frame(self.main_frame)
        self.content_frame.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)
        self.content_frame.rowconfigure(0, weight=1)
        self.content_frame.columnconfigure(0, weight=1) # Video expands
        self.content_frame.columnconfigure(1, weight=0) # Sidebar does not expand
        
        # Video Label
        self.video_label = tk.Label(self.content_frame, bg="black")
        self.video_label.grid(row=0, column=0, sticky="nsew")

        # Sidebar
        self.sidebar_frame = tk.Frame(self.content_frame, width=250, bg="#f0f0f0", relief=tk.SUNKEN, borderwidth=2)
        self.sidebar_frame.grid(row=0, column=1, sticky="ns", padx=(10, 0))
        self.sidebar_frame.pack_propagate(False)

        self.sidebar_title = tk.Label(self.sidebar_frame, text="Registered People", font=("Arial", 16, "bold"), bg="#f0f0f0")
        self.sidebar_title.pack(pady=10, padx=10, anchor="w")

        self.registered_list_var = tk.StringVar()
        self.registered_list_display = tk.Label(self.sidebar_frame, textvariable=self.registered_list_var, font=("Arial", 12), bg="#f0f0f0", justify=tk.LEFT, anchor="nw")
        self.registered_list_display.pack(pady=5, padx=10, fill=tk.X, anchor="nw")
        self.registered_list_var.set("None")

        # --- 2. Bottom Frame (Status & Buttons) ---
        self.bottom_frame = tk.Frame(self.main_frame, bg="#e0e0e0")
        self.bottom_frame.grid(row=1, column=0, sticky="ew", padx=10, pady=(0, 10))
        self.bottom_frame.columnconfigure(0, weight=1) # Status label expands
        self.bottom_frame.columnconfigure(1, weight=0) # Buttons do not expand

        self.status_text = tk.StringVar()
        self.status_text.set("Ready. Press 'Start Camera' to begin.")
        self.status_label = tk.Label(self.bottom_frame, textvariable=self.status_text, font=("Arial", 14), bg="#e0e0e0", anchor="w")
        self.status_label.grid(row=0, column=0, sticky="ew", padx=10, pady=5)
        
        self.button_frame = tk.Frame(self.bottom_frame, bg="#e0e0e0")
        self.button_frame.grid(row=0, column=1, sticky="e", padx=5, pady=5)

        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, font=("Arial", 12), width=15)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, font=("Arial", 12), width=15)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        self.register_button = tk.Button(self.button_frame, text="Register & Monitor", command=self.register_person, font=("Arial", 12), width=18)
        self.register_button.pack(side=tk.LEFT, padx=5)

        self.clear_button = tk.Button(self.button_frame, text="Clear All", command=self.clear_all_registrations, font=("Arial", 12), width=18)
        self.clear_button.pack(side=tk.LEFT, padx=5)

        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        if self.is_running:
            return
        try:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                raise IOError("Cannot open webcam.")
            self.is_running = True
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            self.status_text.set("Camera running. Click 'Register & Monitor'.")
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            if self.cap: self.cap.release()

    def stop_video_stream(self):
        if not self.is_running:
            return
        self.is_running = False
        if self.video_thread:
            self.video_thread.join(timeout=0.5) 
        if self.cap:
            self.cap.release()
        self.video_label.config(image=None)
        self.status_text.set("Camera stopped.")
        self.clear_all_registrations() # Clear people when camera stops

    def video_loop(self):
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    self.status_text.set("Error: Can't read from camera.")
                    time.sleep(0.5)
                    continue

                frame = cv2.flip(frame, 1)
                with self.registration_lock:
                    self.current_frame_for_registration = frame.copy()
                
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                processed_frame, status, list_text = self.process_frame_logic(frame, rgb_frame)
                
                # Update GUI from main thread
                self.status_text.set(status)
                self.registered_list_var.set(list_text if list_text else "None")

                # Resize image
                cv_img = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                w, h = self.video_label.winfo_width(), self.video_label.winfo_height()
                if w > 1 and h > 1:
                     pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)
                
                imgtk = ImageTk.PhotoImage(image=pil_img)
                
                # Update image
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

            except Exception as e:
                print(f"Error in video loop: {e}")
                self.is_running = False
            
            time.sleep(0.01) # ~100fps theoretical max, but processing slows it
        print("Video loop stopped.")

    def get_ear(self, eye_points, w, h):
        """Calculates Eye Aspect Ratio (EAR)."""
        try:
            p1 = (int(eye_points[0].x * w), int(eye_points[0].y * h))
            p4 = (int(eye_points[1].x * w), int(eye_points[1].y * h))
            p2 = (int(eye_points[2].x * w), int(eye_points[2].y * h))
            p6 = (int(eye_points[3].x * w), int(eye_points[3].y * h))
            p3 = (int(eye_points[4].x * w), int(eye_points[4].y * h))
            p5 = (int(eye_points[5].x * w), int(eye_points[5].y * h))
            def get_dist(p_a, p_b):
                return np.linalg.norm(np.array(p_a) - np.array(p_b))
            v_dist_1 = get_dist(p2, p6)
            v_dist_2 = get_dist(p3, p5)
            h_dist = get_dist(p1, p4)
            if h_dist == 0: return 0.3
            ear = (v_dist_1 + v_dist_2) / (2.0 * h_dist)
            return ear
        except Exception:
            return 0.3

    def get_faces_from_results(self, frame, results):
        """Extracts all face data (box, center, EAR) from MediaPipe results."""
        faces_data = []
        h, w, _ = frame.shape
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                x_min, y_min, x_max, y_max = w, h, 0, 0
                for landmark in face_landmarks.landmark:
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    if x < x_min: x_min = x
                    if y < y_min: y_min = y
                    if x > x_max: x_max = x
                    if y > y_max: y_max = y
                padding = 10
                x_min = max(0, x_min - padding)
                y_min = max(0, y_min - padding)
                x_max = min(w, x_max + padding)
                y_max = min(h, y_max + padding)
                box = (x_min, y_min, x_max - x_min, y_max - y_min)
                nose_tip = face_landmarks.landmark[1]
                center = (int(nose_tip.x * w), int(nose_tip.y * h))
                RIGHT_EYE_EAR_POINTS_INDICES = [33, 133, 159, 145, 158, 153]
                LEFT_EYE_EAR_POINTS_INDICES = [362, 263, 386, 374, 385, 380]
                landmarks = face_landmarks.landmark
                right_eye_points = [landmarks[i] for i in RIGHT_EYE_EAR_POINTS_INDICES]
                left_eye_points = [landmarks[i] for i in LEFT_EYE_EAR_POINTS_INDICES]
                right_ear = self.get_ear(right_eye_points, w, h)
                left_ear = self.get_ear(left_eye_points, w, h)
                avg_ear = (left_ear + right_ear) / 2.0
                faces_data.append({'box': box, 'center': center, 'avg_ear': avg_ear})
        return faces_data

    def register_person(self):
        """Registers a new person based on the largest face in the frame."""
        with self.registration_lock:
            if self.current_frame_for_registration is None:
                messagebox.showwarning("Registration Error", "Camera not ready. Please try again.")
                return
            rgb_frame_reg = cv2.cvtColor(self.current_frame_for_registration, cv2.COLOR_BGR2RGB)
            results = self.face_mesh.process(rgb_frame_reg)
            faces = self.get_faces_from_results(self.current_frame_for_registration, results)

        if len(faces) == 0:
            messagebox.showwarning("Registration Error", "No person detected. Please face the camera and try again.")
            return

        faces_by_area = sorted(faces, key=lambda f: f['box'][2] * f['box'][3], reverse=True)
        largest_face = faces_by_area[0]
        current_ear = largest_face['avg_ear']
        
        if current_ear < MIN_OPEN_EYE_EAR:
            messagebox.showwarning("Registration Error", f"Eyes seem to be closed (EAR: {current_ear:.2f}).\nPlease open your eyes and try again.")
            return

        calibrated_threshold = current_ear * THRESHOLD_CALIBRATION_FACTOR
        name = simpledialog.askstring("Register Person", "Enter a unique name:", parent=self.window)
        
        if not name:
            messagebox.showwarning("Registration Cancelled", "Registration was cancelled.")
            return
            
        if name in self.registered_people:
            messagebox.showwarning("Registration Error", f"The name '{name}' is already registered. Please use a unique name.")
            return
        
        # --- Add new person to the dictionary ---
        self.registered_people[name] = {
            'center': largest_face['center'],
            'open_ear': current_ear,
            'threshold': calibrated_threshold,
            'eye_counter': 0,
            'lost_counter': 0,
            'status': 'Calibrated', # For the sidebar list
            'last_alert_time': 0.0 # NEW: For sound loop
        }

        status = f"Calibrated for {name}. Open EAR: {current_ear:.2f}, Threshold: {calibrated_threshold:.2f}"
        self.status_text.set(f"Registered {name}. Monitoring {len(self.registered_people)} person(s).")
        messagebox.showinfo("Registration Complete", status)

    def clear_all_registrations(self):
        """Clears all registered people from monitoring."""
        if not self.registered_people:
            return
        self.registered_people = {}
        self.status_text.set("All registrations cleared. Ready to register.")
        self.registered_list_var.set("None")

    def play_alert_sound(self):
        """Plays the alert sound asynchronously using winsound."""
        if not os.path.exists(ALERT_SOUND_FILE_PATH):
            print(f"Alert Sound Error: File not found at {ALERT_SOUND_FILE_PATH}")
            print("!!! ALERT (SOUND FILE MISSING) !!!")
            return
        try:
            winsound.PlaySound(ALERT_SOUND_FILE_PATH, winsound.SND_FILENAME | winsound.SND_ASYNC)
        except Exception as e:
            print(f"Error playing sound file {ALERT_SOUND_FILE_PATH}: {e}")
            print("!!! ALERT BEEP !!!")

    def process_person(self, frame, person_name, person_data, face_data):
        """
        Processes a single person (found or lost), updates state, and draws on frame.
        """
        alert_triggered = False
        
        if face_data is None:
            # --- Person is LOST ---
            person_data['lost_counter'] += 1
            if person_data['lost_counter'] > LOST_GRACE_FRAMES:
                person_data['status'] = 'Lost'
                person_data['eye_counter'] = 0
            else:
                person_data['status'] = 'Searching...'
            return # Stop processing
        
        # --- Person is FOUND ---
        person_data['lost_counter'] = 0
        person_data['center'] = face_data['center']
        avg_ear = face_data['avg_ear']
        
        # --- Check Eye Status ---
        if avg_ear < person_data['threshold']:
            person_data['eye_counter'] += 1
        else:
            person_data['eye_counter'] = 0
            # Continuous Re-calibration
            alpha = 0.01
            person_data['open_ear'] = (person_data['open_ear'] * (1 - alpha)) + (avg_ear * alpha)
            person_data['threshold'] = person_data['open_ear'] * THRESHOLD_CALIBRATION_FACTOR
        
        # --- Check Alert Status ---
        if person_data['eye_counter'] > EYE_AR_CONSEC_FRAMES:
            person_data['status'] = '!!! ALERT !!!'
            alert_triggered = True
        else:
            person_data['status'] = 'Tracking'
        
        # --- Draw on Frame ---
        (x, y, w, h) = face_data['box']
        
        if alert_triggered:
            color = (0, 0, 255) # Red
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
            cv2.putText(frame, f"ALERT: {person_name} (EYES CLOSED)", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # --- NEW: LOOPING SOUND LOGIC ---
            current_time = time.time()
            if current_time - person_data['last_alert_time'] > ALERT_SOUND_INTERVAL:
                threading.Thread(target=self.play_alert_sound, daemon=True).start()
                person_data['last_alert_time'] = current_time
        else:
            color = (255, 0, 0) # Blue
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            ear_text = f"EAR: {avg_ear:.2f} (Th: {person_data['threshold']:.2f})"
            count_text = f"Count: {person_data['eye_counter']}/{EYE_AR_CONSEC_FRAMES}"
            cv2.putText(frame, ear_text, (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(frame, count_text, (x, y + h + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)


    def process_frame_logic(self, frame, rgb_frame):
        """Main logic loop for processing a frame."""
        results = self.face_mesh.process(rgb_frame)
        faces = self.get_faces_from_results(frame, results)

        if not self.registered_people:
            # Not monitoring anyone, just draw all faces
            for face_data in faces:
                (x, y, w, h) = face_data['box']
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2)
            return frame, "Click 'Register & Monitor' to begin.", ""

        # --- Multi-Person Tracking Logic ---
        unmatched_face_indices = set(range(len(faces)))
        matches = {} # Stores {person_name: face_index}
        
        # Greedily assign best match for each person
        for name, person_data in self.registered_people.items():
            best_dist = float('inf')
            best_face_idx = -1
            for i in unmatched_face_indices: # Only check against unused faces
                dist = np.linalg.norm(np.array(faces[i]['center']) - np.array(person_data['center']))
                if dist < best_dist and dist < MAX_TRACKING_JUMP_PX:
                    best_dist = dist
                    best_face_idx = i
            
            if best_face_idx != -1:
                matches[name] = best_face_idx
                unmatched_face_indices.remove(best_face_idx) # This face is now taken

        list_display_text = []
        # Process all people (matched or lost)
        for name, person_data in self.registered_people.items():
            face_data = None
            if name in matches:
                face_data = faces[matches[name]] # Get the matched face data
            
            # This function updates the person's state and draws on the frame
            self.process_person(frame, name, person_data, face_data)
            
            list_display_text.append(f"{name}: {person_data['status']}")

        # Draw remaining unmatched (unregistered) faces
        for i in unmatched_face_indices:
            (x, y, w, h) = faces[i]['box']
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2)

        status = f"Monitoring {len(self.registered_people)} person(s)."
        return frame, status, "\n".join(list_display_text)

    def on_closing(self):
        """Handles the window close event."""
        if messagebox.askokcancel("Quit", "Do you want to exit the application?"):
            self.stop_video_stream()
            self.window.destroy()

# --- Main execution ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except ImportError:
        print("\n--- ERROR ---")
        print("Could not import one or more required libraries.")
        print("Please ensure you have the following installed:")
        print("pip install opencv-python pillow numpy mediapipe")
        print("---------------")
    except Exception as e:
        print(f"An error occurred: {e}")

: 

In [1]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os
import mediapipe as mp  # --- Import MediaPipe ---
import winsound        # --- Using built-in winsound for Windows ---

# --- Configuration Constants ---

# --- Path to the alert sound ---
# Make sure to use double backslashes (\\) on Windows!
ALERT_SOUND_FILE_PATH = "D:\\GIT_HUB\\12_Final_Projects_of_all\\03_deep_learning\\beep-01a.wav"

# --- Sound loop configuration ---
# Time in seconds between playing the alert sound
ALERT_SOUND_INTERVAL = 2.0 # Play sound every 2 seconds

# --- Dynamic Calibration Constants ---
THRESHOLD_CALIBRATION_FACTOR = 0.75
MIN_OPEN_EYE_EAR = 0.20
EYE_AR_CONSEC_FRAMES = 200 # ~10 seconds at 20fps

# --- Tracking Constants ---
LOST_GRACE_FRAMES = 50 # ~2.5 seconds
MAX_TRACKING_JUMP_PX = 150 # Max distance a face can move between frames

class SleepingAlertApp:
    def __init__(self, window):
        """
        Initialize the application.
        """
        self.window = window
        self.window.title("Multi-Person Sleeping Alert System (v2.1 - Stable Calibration)")
        self.window.geometry("1000x700") # Made window wider for sidebar
        self.window.minsize(800, 600) # Set a minimum size

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- MediaPipe Face Mesh Initialization ---
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=10,  # Detect more faces
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # --- Multi-Person Logic & Registration State ---
        self.registered_people = {} # Main dictionary to hold all registered people
        self.registration_lock = threading.Lock()
        self.current_frame_for_registration = None

        # --- GUI REBUILD: Using .grid() for stable layout ---
        
        # Configure the main window's grid
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        self.main_frame.rowconfigure(0, weight=1) # Content (video/sidebar) expands
        self.main_frame.rowconfigure(1, weight=0) # Bottom bar does not expand
        self.main_frame.columnconfigure(0, weight=1) # Main column expands
        
        # --- 1. Content Frame (Video & Sidebar) ---
        self.content_frame = tk.Frame(self.main_frame)
        self.content_frame.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)
        self.content_frame.rowconfigure(0, weight=1)
        self.content_frame.columnconfigure(0, weight=1) # Video expands
        self.content_frame.columnconfigure(1, weight=0) # Sidebar does not expand
        
        # Video Label
        self.video_label = tk.Label(self.content_frame, bg="black")
        self.video_label.grid(row=0, column=0, sticky="nsew")

        # Sidebar
        self.sidebar_frame = tk.Frame(self.content_frame, width=250, bg="#f0f0f0", relief=tk.SUNKEN, borderwidth=2)
        self.sidebar_frame.grid(row=0, column=1, sticky="ns", padx=(10, 0))
        self.sidebar_frame.pack_propagate(False)

        self.sidebar_title = tk.Label(self.sidebar_frame, text="Registered People", font=("Arial", 16, "bold"), bg="#f0f0f0")
        self.sidebar_title.pack(pady=10, padx=10, anchor="w")

        self.registered_list_var = tk.StringVar()
        self.registered_list_display = tk.Label(self.sidebar_frame, textvariable=self.registered_list_var, font=("Arial", 12), bg="#f0f0f0", justify=tk.LEFT, anchor="nw")
        self.registered_list_display.pack(pady=5, padx=10, fill=tk.X, anchor="nw")
        self.registered_list_var.set("None")

        # --- 2. Bottom Frame (Status & Buttons) ---
        self.bottom_frame = tk.Frame(self.main_frame, bg="#e0e0e0")
        self.bottom_frame.grid(row=1, column=0, sticky="ew", padx=10, pady=(0, 10))
        self.bottom_frame.columnconfigure(0, weight=1) # Status label expands
        self.bottom_frame.columnconfigure(1, weight=0) # Buttons do not expand

        self.status_text = tk.StringVar()
        self.status_text.set("Ready. Press 'Start Camera' to begin.")
        self.status_label = tk.Label(self.bottom_frame, textvariable=self.status_text, font=("Arial", 14), bg="#e0e0e0", anchor="w")
        self.status_label.grid(row=0, column=0, sticky="ew", padx=10, pady=5)
        
        self.button_frame = tk.Frame(self.bottom_frame, bg="#e0e0e0")
        self.button_frame.grid(row=0, column=1, sticky="e", padx=5, pady=5)

        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, font=("Arial", 12), width=15)
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, font=("Arial", 12), width=15)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        self.register_button = tk.Button(self.button_frame, text="Register & Monitor", command=self.register_person, font=("Arial", 12), width=18)
        self.register_button.pack(side=tk.LEFT, padx=5)

        self.clear_button = tk.Button(self.button_frame, text="Clear All", command=self.clear_all_registrations, font=("Arial", 12), width=18)
        self.clear_button.pack(side=tk.LEFT, padx=5)

        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        if self.is_running:
            return
        try:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                raise IOError("Cannot open webcam.")
            self.is_running = True
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            self.status_text.set("Camera running. Click 'Register & Monitor'.")
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            if self.cap: self.cap.release()

    def stop_video_stream(self):
        if not self.is_running:
            return
        self.is_running = False
        if self.video_thread:
            self.video_thread.join(timeout=0.5) 
        if self.cap:
            self.cap.release()
        self.video_label.config(image=None)
        self.status_text.set("Camera stopped.")
        self.clear_all_registrations() # Clear people when camera stops

    def video_loop(self):
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    self.status_text.set("Error: Can't read from camera.")
                    time.sleep(0.5)
                    continue

                frame = cv2.flip(frame, 1)
                with self.registration_lock:
                    self.current_frame_for_registration = frame.copy()
                
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                processed_frame, status, list_text = self.process_frame_logic(frame, rgb_frame)
                
                # Update GUI from main thread
                self.status_text.set(status)
                self.registered_list_var.set(list_text if list_text else "None")

                # Resize image
                cv_img = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                w, h = self.video_label.winfo_width(), self.video_label.winfo_height()
                if w > 1 and h > 1:
                     pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)
                
                imgtk = ImageTk.PhotoImage(image=pil_img)
                
                # Update image
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

            except Exception as e:
                print(f"Error in video loop: {e}")
                self.is_running = False
            
            time.sleep(0.01) # ~100fps theoretical max, but processing slows it
        print("Video loop stopped.")

    def get_ear(self, eye_points, w, h):
        """Calculates Eye Aspect Ratio (EAR)."""
        try:
            p1 = (int(eye_points[0].x * w), int(eye_points[0].y * h))
            p4 = (int(eye_points[1].x * w), int(eye_points[1].y * h))
            p2 = (int(eye_points[2].x * w), int(eye_points[2].y * h))
            p6 = (int(eye_points[3].x * w), int(eye_points[3].y * h))
            p3 = (int(eye_points[4].x * w), int(eye_points[4].y * h))
            p5 = (int(eye_points[5].x * w), int(eye_points[5].y * h))
            def get_dist(p_a, p_b):
                return np.linalg.norm(np.array(p_a) - np.array(p_b))
            v_dist_1 = get_dist(p2, p6)
            v_dist_2 = get_dist(p3, p5)
            h_dist = get_dist(p1, p4)
            if h_dist == 0: return 0.3
            ear = (v_dist_1 + v_dist_2) / (2.0 * h_dist)
            return ear
        except Exception:
            return 0.3

    def get_faces_from_results(self, frame, results):
        """Extracts all face data (box, center, EAR) from MediaPipe results."""
        faces_data = []
        h, w, _ = frame.shape
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                x_min, y_min, x_max, y_max = w, h, 0, 0
                for landmark in face_landmarks.landmark:
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    if x < x_min: x_min = x
                    if y < y_min: y_min = y
                    if x > x_max: x_max = x
                    if y > y_max: y_max = y
                padding = 10
                x_min = max(0, x_min - padding)
                y_min = max(0, y_min - padding)
                x_max = min(w, x_max + padding)
                y_max = min(h, y_max + padding)
                box = (x_min, y_min, x_max - x_min, y_max - y_min)
                nose_tip = face_landmarks.landmark[1]
                center = (int(nose_tip.x * w), int(nose_tip.y * h))
                RIGHT_EYE_EAR_POINTS_INDICES = [33, 133, 159, 145, 158, 153]
                LEFT_EYE_EAR_POINTS_INDICES = [362, 263, 386, 374, 385, 380]
                landmarks = face_landmarks.landmark
                right_eye_points = [landmarks[i] for i in RIGHT_EYE_EAR_POINTS_INDICES]
                left_eye_points = [landmarks[i] for i in LEFT_EYE_EAR_POINTS_INDICES]
                right_ear = self.get_ear(right_eye_points, w, h)
                left_ear = self.get_ear(left_eye_points, w, h)
                avg_ear = (left_ear + right_ear) / 2.0
                faces_data.append({'box': box, 'center': center, 'avg_ear': avg_ear})
        return faces_data

    def register_person(self):
        """Registers a new person based on the largest face in the frame."""
        with self.registration_lock:
            if self.current_frame_for_registration is None:
                messagebox.showwarning("Registration Error", "Camera not ready. Please try again.")
                return
            rgb_frame_reg = cv2.cvtColor(self.current_frame_for_registration, cv2.COLOR_BGR2RGB)
            results = self.face_mesh.process(rgb_frame_reg)
            faces = self.get_faces_from_results(self.current_frame_for_registration, results)

        if len(faces) == 0:
            messagebox.showwarning("Registration Error", "No person detected. Please face the camera and try again.")
            return

        faces_by_area = sorted(faces, key=lambda f: f['box'][2] * f['box'][3], reverse=True)
        largest_face = faces_by_area[0]
        current_ear = largest_face['avg_ear']
        
        if current_ear < MIN_OPEN_EYE_EAR:
            messagebox.showwarning("Registration Error", f"Eyes seem to be closed (EAR: {current_ear:.2f}).\nPlease open your eyes and try again.")
            return

        calibrated_threshold = current_ear * THRESHOLD_CALIBRATION_FACTOR
        name = simpledialog.askstring("Register Person", "Enter a unique name:", parent=self.window)
        
        if not name:
            messagebox.showwarning("Registration Cancelled", "Registration was cancelled.")
            return
            
        if name in self.registered_people:
            messagebox.showwarning("Registration Error", f"The name '{name}' is already registered. Please use a unique name.")
            return
        
        # --- Add new person to the dictionary ---
        self.registered_people[name] = {
            'center': largest_face['center'],
            'open_ear': current_ear,
            'threshold': calibrated_threshold,
            'eye_counter': 0,
            'lost_counter': 0,
            'status': 'Calibrated', # For the sidebar list
            'last_alert_time': 0.0 # For sound loop
        }

        status = f"Calibrated for {name}. Open EAR: {current_ear:.2f}, Threshold: {calibrated_threshold:.2f}"
        self.status_text.set(f"Registered {name}. Monitoring {len(self.registered_people)} person(s).")
        messagebox.showinfo("Registration Complete", status)

    def clear_all_registrations(self):
        """Clears all registered people from monitoring."""
        if not self.registered_people:
            return
        self.registered_people = {}
        self.status_text.set("All registrations cleared. Ready to register.")
        self.registered_list_var.set("None")

    def play_alert_sound(self):
        """Plays the alert sound asynchronously using winsound."""
        if not os.path.exists(ALERT_SOUND_FILE_PATH):
            print(f"Alert Sound Error: File not found at {ALERT_SOUND_FILE_PATH}")
            print("!!! ALERT (SOUND FILE MISSING) !!!")
            return
        try:
            winsound.PlaySound(ALERT_SOUND_FILE_PATH, winsound.SND_FILENAME | winsound.SND_ASYNC)
        except Exception as e:
            print(f"Error playing sound file {ALERT_SOUND_FILE_PATH}: {e}")
            print("!!! ALERT BEEP !!!")

    def process_person(self, frame, person_name, person_data, face_data):
        """
        Processes a single person (found or lost), updates state, and draws on frame.
        """
        alert_triggered = False
        
        if face_data is None:
            # --- Person is LOST ---
            person_data['lost_counter'] += 1
            if person_data['lost_counter'] > LOST_GRACE_FRAMES:
                person_data['status'] = 'Lost'
                person_data['eye_counter'] = 0
            else:
                person_data['status'] = 'Searching...'
            return # Stop processing
        
        # --- Person is FOUND ---
        person_data['lost_counter'] = 0
        person_data['center'] = face_data['center']
        avg_ear = face_data['avg_ear']
        
        # --- Check Eye Status ---
        if avg_ear < person_data['threshold']:
            person_data['eye_counter'] += 1
        else:
            person_data['eye_counter'] = 0
            
            # --- LOGIC FIX: REMOVED continuous re-calibration ---
            # The calibration is now stable and set only at registration.
            # This prevents the system from "learning" a sleepy state.
        
        # --- Check Alert Status ---
        if person_data['eye_counter'] > EYE_AR_CONSEC_FRAMES:
            person_data['status'] = '!!! ALERT !!!'
            alert_triggered = True
        else:
            person_data['status'] = 'Tracking'
        
        # --- Draw on Frame ---
        (x, y, w, h) = face_data['box']
        
        if alert_triggered:
            color = (0, 0, 255) # Red
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
            cv2.putText(frame, f"ALERT: {person_name} (EYES CLOSED)", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            
            # --- LOOPING SOUND LOGIC ---
            current_time = time.time()
            if current_time - person_data['last_alert_time'] > ALERT_SOUND_INTERVAL:
                threading.Thread(target=self.play_alert_sound, daemon=True).start()
                person_data['last_alert_time'] = current_time
        else:
            color = (255, 0, 0) # Blue
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            ear_text = f"EAR: {avg_ear:.2f} (Th: {person_data['threshold']:.2f})"
            count_text = f"Count: {person_data['eye_counter']}/{EYE_AR_CONSEC_FRAMES}"
            cv2.putText(frame, ear_text, (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(frame, count_text, (x, y + h + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)


    def process_frame_logic(self, frame, rgb_frame):
        """Main logic loop for processing a frame."""
        results = self.face_mesh.process(rgb_frame)
        faces = self.get_faces_from_results(frame, results)

        if not self.registered_people:
            # Not monitoring anyone, just draw all faces
            for face_data in faces:
                (x, y, w, h) = face_data['box']
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2)
            return frame, "Click 'Register & Monitor' to begin.", ""

        # --- Multi-Person Tracking Logic ---
        unmatched_face_indices = set(range(len(faces)))
        matches = {} # Stores {person_name: face_index}
        
        # Greedily assign best match for each person
        for name, person_data in self.registered_people.items():
            best_dist = float('inf')
            best_face_idx = -1
            for i in unmatched_face_indices: # Only check against unused faces
                dist = np.linalg.norm(np.array(faces[i]['center']) - np.array(person_data['center']))
                if dist < best_dist and dist < MAX_TRACKING_JUMP_PX:
                    best_dist = dist
                    best_face_idx = i
            
            if best_face_idx != -1:
                matches[name] = best_face_idx
                unmatched_face_indices.remove(best_face_idx) # This face is now taken

        list_display_text = []
        # Process all people (matched or lost)
        for name, person_data in self.registered_people.items():
            face_data = None
            if name in matches:
                face_data = faces[matches[name]] # Get the matched face data
            
            # This function updates the person's state and draws on the frame
            self.process_person(frame, name, person_data, face_data)
            
            list_display_text.append(f"{name}: {person_data['status']}")

        # Draw remaining unmatched (unregistered) faces
        for i in unmatched_face_indices:
            (x, y, w, h) = faces[i]['box']
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 192, 0), 2)

        status = f"Monitoring {len(self.registered_people)} person(s)."
        return frame, status, "\n".join(list_display_text)

    def on_closing(self):
        """Handles the window close event."""
        if messagebox.askokcancel("Quit", "Do you want to exit the application?"):
            self.stop_video_stream()
            self.window.destroy()

# --- Main execution ---
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except ImportError:
        print("\n--- ERROR ---")
        print("Could not import one or more required libraries.")
        print("Please ensure you have the following installed:")
        print("pip install opencv-python pillow numpy mediapipe")
        print("---------------")
    except Exception as e:
        print(f"An error occurred: {e}")

Video loop stopped.


In [1]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os
import logging  # --- NEW: For Logging ---
import mediapipe as mp
import winsound

# --- Logging Setup ---
# This creates a file named 'sleeping_system_log.txt' and appends events to it.
logging.basicConfig(
    filename='sleeping_system_log.txt',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# --- Configuration Constants ---
ALERT_SOUND_FILE_PATH = "D:\\GIT_HUB\\12_Final_Projects_of_all\\03_deep_learning\\beep-01a.wav"
ALERT_SOUND_INTERVAL = 2.0

# --- Improved Logic Constants ---
# Lowered factor to 0.65 to prevent false positives when eyes are just "relaxed"
THRESHOLD_CALIBRATION_FACTOR = 0.65 
# Hard limit: Threshold will never be set higher than this, even if eyes are huge.
MAX_POSSIBLE_THRESHOLD = 0.28 
MIN_OPEN_EYE_EAR = 0.18
EYE_AR_CONSEC_FRAMES = 150 # Slightly faster reaction (~7-8 seconds)

# --- Calibration Constants ---
CALIBRATION_FRAMES_REQUIRED = 40 # Number of frames to average for robust calibration

# --- Tracking Constants ---
LOST_GRACE_FRAMES = 50
MAX_TRACKING_JUMP_PX = 150

class SleepingAlertApp:
    def __init__(self, window):
        self.window = window
        self.window.title("Sleeping Alert System v3.0 (Logs + Burst Calibration)")
        self.window.geometry("1100x750")
        self.window.minsize(900, 650)

        logging.info("Application started.")

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- MediaPipe Setup ---
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=5,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # --- Multi-Person & Logic State ---
        self.registered_people = {}
        self.registration_lock = threading.Lock()
        self.current_frame_for_registration = None
        
        # --- NEW: Calibration State ---
        self.is_calibrating = False
        self.calibration_buffer = [] # Stores EAR values during calibration
        self.calibration_face_center = None # Tracks who we are calibrating

        # --- GUI Setup ---
        self._setup_gui()

    def _setup_gui(self):
        """Helper to organize GUI code."""
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        self.main_frame.rowconfigure(0, weight=1)
        self.main_frame.rowconfigure(1, weight=0)
        self.main_frame.columnconfigure(0, weight=1)
        
        # 1. Content Area
        self.content_frame = tk.Frame(self.main_frame)
        self.content_frame.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)
        self.content_frame.rowconfigure(0, weight=1)
        self.content_frame.columnconfigure(0, weight=1)
        self.content_frame.columnconfigure(1, weight=0)
        
        self.video_label = tk.Label(self.content_frame, bg="black")
        self.video_label.grid(row=0, column=0, sticky="nsew")

        # Sidebar
        self.sidebar_frame = tk.Frame(self.content_frame, width=280, bg="#f0f0f0", relief=tk.SUNKEN, borderwidth=2)
        self.sidebar_frame.grid(row=0, column=1, sticky="ns", padx=(10, 0))
        self.sidebar_frame.pack_propagate(False)

        tk.Label(self.sidebar_frame, text="Registered People", font=("Arial", 14, "bold"), bg="#f0f0f0").pack(pady=10, anchor="w", padx=10)
        
        self.registered_list_var = tk.StringVar(value="No one registered.")
        self.registered_list_display = tk.Label(self.sidebar_frame, textvariable=self.registered_list_var, font=("Arial", 11), bg="#f0f0f0", justify=tk.LEFT, anchor="nw")
        self.registered_list_display.pack(pady=5, padx=10, fill=tk.X, anchor="nw")

        # 2. Bottom Control Bar
        self.bottom_frame = tk.Frame(self.main_frame, bg="#e0e0e0")
        self.bottom_frame.grid(row=1, column=0, sticky="ew", padx=10, pady=(0, 10))
        self.bottom_frame.columnconfigure(0, weight=1)

        self.status_text = tk.StringVar(value="Ready. Press 'Start Camera'.")
        self.status_label = tk.Label(self.bottom_frame, textvariable=self.status_text, font=("Arial", 12, "bold"), bg="#e0e0e0", anchor="w")
        self.status_label.grid(row=0, column=0, sticky="ew", padx=10, pady=10)
        
        self.button_frame = tk.Frame(self.bottom_frame, bg="#e0e0e0")
        self.button_frame.grid(row=0, column=1, sticky="e", padx=5)

        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, width=15, bg="#dddddd")
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, width=15, bg="#dddddd", state=tk.DISABLED)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        self.register_button = tk.Button(self.button_frame, text="Register & Monitor", command=self.initiate_calibration, width=20, bg="#aaffaa", state=tk.DISABLED)
        self.register_button.pack(side=tk.LEFT, padx=5)

        self.clear_button = tk.Button(self.button_frame, text="Clear All", command=self.clear_all_registrations, width=15, bg="#ffaaaa", state=tk.DISABLED)
        self.clear_button.pack(side=tk.LEFT, padx=5)

        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        if self.is_running: return
        try:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened(): raise IOError("Cannot open webcam.")
            
            self.is_running = True
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            
            self.start_button.config(state=tk.DISABLED)
            self.stop_button.config(state=tk.NORMAL)
            self.register_button.config(state=tk.NORMAL)
            self.clear_button.config(state=tk.NORMAL)
            self.status_text.set("Camera running. Align face and click 'Register'.")
            logging.info("Camera started.")
            
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            logging.error(f"Webcam failed to start: {e}")

    def stop_video_stream(self):
        if not self.is_running: return
        self.is_running = False
        if self.video_thread: self.video_thread.join(timeout=0.5) 
        if self.cap: self.cap.release()
        
        self.video_label.config(image=None)
        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        self.register_button.config(state=tk.DISABLED)
        self.clear_button.config(state=tk.DISABLED)
        self.status_text.set("Camera stopped.")
        
        self.clear_all_registrations()
        logging.info("Camera stopped.")

    def video_loop(self):
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    time.sleep(0.1)
                    continue

                frame = cv2.flip(frame, 1)
                
                # Handle Calibration Logic inside the loop for accuracy
                if self.is_calibrating:
                    frame = self._handle_calibration_step(frame)
                else:
                    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    frame, status_msg, list_msg = self.process_monitoring_logic(frame, rgb_frame)
                    
                    # Update UI safely
                    self.status_text.set(status_msg)
                    self.registered_list_var.set(list_msg)

                # Display Frame
                cv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                
                # Responsive resize
                w = self.video_label.winfo_width()
                h = self.video_label.winfo_height()
                if w > 10 and h > 10:
                    pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)
                
                imgtk = ImageTk.PhotoImage(image=pil_img)
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

            except Exception as e:
                print(f"Loop Error: {e}")
                logging.error(f"Video loop error: {e}")
                self.is_running = False
            
            time.sleep(0.01)

    # --- Core Logic Methods ---

    def get_ear(self, eye_points, w, h):
        """Calculates Eye Aspect Ratio."""
        try:
            # Standard Euclidean Distance
            def dist(p1, p2):
                return np.linalg.norm(np.array([p1.x*w, p1.y*h]) - np.array([p2.x*w, p2.y*h]))

            # Vertical distances
            v1 = dist(eye_points[1], eye_points[5])
            v2 = dist(eye_points[2], eye_points[4])
            # Horizontal distance
            h_dist = dist(eye_points[0], eye_points[3])

            if h_dist == 0: return 0.0
            return (v1 + v2) / (2.0 * h_dist)
        except:
            return 0.0

    def get_face_data(self, frame, results):
        """Parses MediaPipe results into usable data (box, EAR, center)."""
        faces = []
        h, w, _ = frame.shape
        if not results.multi_face_landmarks: return faces

        for landmarks in results.multi_face_landmarks:
            # Bounding Box
            xs = [l.x for l in landmarks.landmark]
            ys = [l.y for l in landmarks.landmark]
            x_min, x_max = int(min(xs)*w), int(max(xs)*w)
            y_min, y_max = int(min(ys)*h), int(max(ys)*h)
            
            # Expand box slightly
            x_min, y_min = max(0, x_min-10), max(0, y_min-10)
            x_max, y_max = min(w, x_max+10), min(h, y_max+10)
            
            # Nose Tip (Landmark 1)
            nose = landmarks.landmark[1]
            center = (int(nose.x*w), int(nose.y*h))

            # EAR Calculation
            # Indices: Left [33, 160, 158, 133, 153, 144], Right [362, 385, 387, 263, 373, 380]
            # Using the indices from previous iteration that were verified:
            LEFT_IDXS = [33, 159, 158, 133, 153, 145] # P1, P2, P3, P4, P5, P6
            RIGHT_IDXS = [362, 386, 385, 263, 380, 374]

            left_pts = [landmarks.landmark[i] for i in LEFT_IDXS]
            right_pts = [landmarks.landmark[i] for i in RIGHT_IDXS]

            ear_left = self.get_ear(left_pts, w, h)
            ear_right = self.get_ear(right_pts, w, h)
            avg_ear = (ear_left + ear_right) / 2.0

            faces.append({
                'box': (x_min, y_min, x_max-x_min, y_max-y_min),
                'center': center,
                'ear': avg_ear
            })
        return faces

    # --- NEW: Robust Calibration Workflow ---

    def initiate_calibration(self):
        """Starts the burst calibration process."""
        if self.is_calibrating: return
        self.is_calibrating = True
        self.calibration_buffer = []
        self.status_text.set("CALIBRATING... KEEP EYES OPEN AND STILL!")
        # Disable buttons
        self.register_button.config(state=tk.DISABLED)
        logging.info("Calibration initiated.")

    def _handle_calibration_step(self, frame):
        """Accumulates frames to calculate average EAR."""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(rgb_frame)
        faces = self.get_face_data(frame, results)
        
        h, w, _ = frame.shape
        
        if not faces:
            cv2.putText(frame, "NO FACE DETECTED", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            return frame

        # Find largest face
        target = sorted(faces, key=lambda f: f['box'][2]*f['box'][3], reverse=True)[0]
        
        # Draw Loading Bar
        progress = len(self.calibration_buffer) / CALIBRATION_FRAMES_REQUIRED
        bar_w = int(w * 0.6)
        cv2.rectangle(frame, (int(w*0.2), h-100), (int(w*0.2) + bar_w, h-70), (255, 255, 255), 2)
        cv2.rectangle(frame, (int(w*0.2), h-100), (int(w*0.2) + int(bar_w*progress), h-70), (0, 255, 0), -1)
        cv2.putText(frame, "CALIBRATING...", (int(w*0.2), h-110), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        self.calibration_buffer.append(target['ear'])

        if len(self.calibration_buffer) >= CALIBRATION_FRAMES_REQUIRED:
            self._finalize_calibration(target['center'])
            self.is_calibrating = False
            self.register_button.config(state=tk.NORMAL)

        return frame

    def _finalize_calibration(self, face_center):
        """Calculates the final threshold from the buffer."""
        avg_ear = sum(self.calibration_buffer) / len(self.calibration_buffer)
        
        # Logic Check
        if avg_ear < MIN_OPEN_EYE_EAR:
            messagebox.showwarning("Failed", f"Eyes detected as closed (EAR: {avg_ear:.2f}). Retry.")
            logging.warning(f"Calibration failed: EAR too low ({avg_ear:.2f})")
            return

        # Calculate robust threshold
        threshold = min(avg_ear * THRESHOLD_CALIBRATION_FACTOR, MAX_POSSIBLE_THRESHOLD)
        
        # Get Name (must be done in main thread, but simpledialog works here usually)
        name = simpledialog.askstring("Registration", f"Calibration Done (EAR: {avg_ear:.2f}).\nName:")
        
        if name and name not in self.registered_people:
            self.registered_people[name] = {
                'center': face_center,
                'threshold': threshold,
                'open_ear_baseline': avg_ear,
                'closed_frames': 0,
                'status': 'Active',
                'last_sound_time': 0
            }
            logging.info(f"Person registered: {name} with threshold {threshold:.3f}")
        else:
            logging.warning("Registration cancelled or duplicate name.")

    # --- Monitoring Logic ---

    def process_monitoring_logic(self, frame, rgb_frame):
        results = self.face_mesh.process(rgb_frame)
        faces = self.get_face_data(frame, results)
        
        if not self.registered_people:
            # Just draw boxes if no one registered
            for f in faces:
                x, y, w, h = f['box']
                cv2.rectangle(frame, (x,y), (x+w, y+h), (100, 100, 100), 2)
            return frame, "Ready to Register.", "No one registered."

        # Matching Logic (Simple Nearest Neighbor)
        active_faces = {i: f for i, f in enumerate(faces)}
        matched_names = []
        list_display = []

        for name, p_data in self.registered_people.items():
            best_dist = float('inf')
            best_idx = -1
            
            # Find closest face
            for idx, face in active_faces.items():
                dist = np.linalg.norm(np.array(face['center']) - np.array(p_data['center']))
                if dist < best_dist:
                    best_dist = dist
                    best_idx = idx
            
            # Check if match is valid (tracking continuity)
            if best_idx != -1 and best_dist < MAX_TRACKING_JUMP_PX:
                # Found the person
                face = active_faces[best_idx]
                del active_faces[best_idx] # Remove from pool
                
                # Update position for next frame
                p_data['center'] = face['center']
                
                # --- SLEEP DETECTION LOGIC ---
                ear = face['ear']
                is_eyes_closed = ear < p_data['threshold']
                
                if is_eyes_closed:
                    p_data['closed_frames'] += 1
                else:
                    p_data['closed_frames'] = 0 # Reset immediately on open eyes
                
                # Trigger Alert
                if p_data['closed_frames'] > EYE_AR_CONSEC_FRAMES:
                    p_data['status'] = "!!! SLEEPING !!!"
                    self._trigger_alert(name, p_data)
                    color = (0, 0, 255)
                    cv2.putText(frame, "WAKE UP!", (face['box'][0], face['box'][1]-20), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)
                else:
                    p_data['status'] = "Active"
                    color = (0, 255, 0)

                # Draw Info
                x, y, w, h = face['box']
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.putText(frame, f"{name} (EAR: {ear:.2f})", (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
            else:
                # Person lost
                p_data['status'] = "Lost"
                p_data['closed_frames'] = 0

            list_display.append(f"{name}: {p_data['status']}")

        status_msg = f"Monitoring {len(self.registered_people)} people."
        return frame, status_msg, "\n".join(list_display)

    def _trigger_alert(self, name, p_data):
        """Plays sound loop and logs alert."""
        now = time.time()
        if now - p_data['last_sound_time'] > ALERT_SOUND_INTERVAL:
            logging.warning(f"Alert triggered for {name}!")
            threading.Thread(target=self._play_sound, daemon=True).start()
            p_data['last_sound_time'] = now

    def _play_sound(self):
        if os.path.exists(ALERT_SOUND_FILE_PATH):
            try:
                winsound.PlaySound(ALERT_SOUND_FILE_PATH, winsound.SND_FILENAME | winsound.SND_ASYNC)
            except:
                pass # Fail silently in thread

    def clear_all_registrations(self):
        self.registered_people = {}
        logging.info("All registrations cleared.")
        self.status_text.set("Registrations Cleared.")

    def on_closing(self):
        if messagebox.askokcancel("Quit", "Exit Application?"):
            self.stop_video_stream()
            logging.info("Application closed.")
            self.window.destroy()

if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except Exception as e:
        logging.critical(f"Critical crash: {e}")
        print(f"Crash: {e}")

Loop Error: window ".!_querystring" was deleted before its visibility changed


In [1]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import time
import os
import logging
import mediapipe as mp
import winsound

# --- Logging Setup ---
logging.basicConfig(
    filename='sleeping_system_log.txt',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# --- Configuration Constants ---
ALERT_SOUND_FILE_PATH = "D:\\GIT_HUB\\12_Final_Projects_of_all\\03_deep_learning\\beep-01a.wav"
ALERT_SOUND_INTERVAL = 2.0

# --- Improved Logic Constants ---
THRESHOLD_CALIBRATION_FACTOR = 0.65 
MAX_POSSIBLE_THRESHOLD = 0.28 
MIN_OPEN_EYE_EAR = 0.18
EYE_AR_CONSEC_FRAMES = 150 

# --- Calibration Constants ---
CALIBRATION_FRAMES_REQUIRED = 40 

# --- Tracking Constants ---
LOST_GRACE_FRAMES = 50
MAX_TRACKING_JUMP_PX = 150

class SleepingAlertApp:
    def __init__(self, window):
        self.window = window
        self.window.title("Sleeping Alert System v3.1 (Thread Safe)")
        self.window.geometry("1100x750")
        self.window.minsize(900, 650)

        logging.info("Application started.")

        # --- State Variables ---
        self.cap = None
        self.video_thread = None
        self.is_running = False

        # --- MediaPipe Setup ---
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            max_num_faces=5,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        # --- Multi-Person & Logic State ---
        self.registered_people = {}
        self.registration_lock = threading.Lock()
        self.current_frame_for_registration = None
        
        # --- NEW: Calibration State ---
        self.is_calibrating = False
        self.calibration_buffer = [] 
        
        # --- GUI Setup ---
        self._setup_gui()

    def _setup_gui(self):
        """Helper to organize GUI code."""
        self.main_frame = tk.Frame(self.window)
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        self.main_frame.rowconfigure(0, weight=1)
        self.main_frame.rowconfigure(1, weight=0)
        self.main_frame.columnconfigure(0, weight=1)
        
        # 1. Content Area
        self.content_frame = tk.Frame(self.main_frame)
        self.content_frame.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)
        self.content_frame.rowconfigure(0, weight=1)
        self.content_frame.columnconfigure(0, weight=1)
        self.content_frame.columnconfigure(1, weight=0)
        
        self.video_label = tk.Label(self.content_frame, bg="black")
        self.video_label.grid(row=0, column=0, sticky="nsew")

        # Sidebar
        self.sidebar_frame = tk.Frame(self.content_frame, width=280, bg="#f0f0f0", relief=tk.SUNKEN, borderwidth=2)
        self.sidebar_frame.grid(row=0, column=1, sticky="ns", padx=(10, 0))
        self.sidebar_frame.pack_propagate(False)

        tk.Label(self.sidebar_frame, text="Registered People", font=("Arial", 14, "bold"), bg="#f0f0f0").pack(pady=10, anchor="w", padx=10)
        
        self.registered_list_var = tk.StringVar(value="No one registered.")
        self.registered_list_display = tk.Label(self.sidebar_frame, textvariable=self.registered_list_var, font=("Arial", 11), bg="#f0f0f0", justify=tk.LEFT, anchor="nw")
        self.registered_list_display.pack(pady=5, padx=10, fill=tk.X, anchor="nw")

        # 2. Bottom Control Bar
        self.bottom_frame = tk.Frame(self.main_frame, bg="#e0e0e0")
        self.bottom_frame.grid(row=1, column=0, sticky="ew", padx=10, pady=(0, 10))
        self.bottom_frame.columnconfigure(0, weight=1)

        self.status_text = tk.StringVar(value="Ready. Press 'Start Camera'.")
        self.status_label = tk.Label(self.bottom_frame, textvariable=self.status_text, font=("Arial", 12, "bold"), bg="#e0e0e0", anchor="w")
        self.status_label.grid(row=0, column=0, sticky="ew", padx=10, pady=10)
        
        self.button_frame = tk.Frame(self.bottom_frame, bg="#e0e0e0")
        self.button_frame.grid(row=0, column=1, sticky="e", padx=5)

        self.start_button = tk.Button(self.button_frame, text="Start Camera", command=self.start_video_stream, width=15, bg="#dddddd")
        self.start_button.pack(side=tk.LEFT, padx=5)

        self.stop_button = tk.Button(self.button_frame, text="Stop Camera", command=self.stop_video_stream, width=15, bg="#dddddd", state=tk.DISABLED)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        self.register_button = tk.Button(self.button_frame, text="Register & Monitor", command=self.initiate_calibration, width=20, bg="#aaffaa", state=tk.DISABLED)
        self.register_button.pack(side=tk.LEFT, padx=5)

        self.clear_button = tk.Button(self.button_frame, text="Clear All", command=self.clear_all_registrations, width=15, bg="#ffaaaa", state=tk.DISABLED)
        self.clear_button.pack(side=tk.LEFT, padx=5)

        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def start_video_stream(self):
        if self.is_running: return
        try:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened(): raise IOError("Cannot open webcam.")
            
            self.is_running = True
            self.video_thread = threading.Thread(target=self.video_loop, daemon=True)
            self.video_thread.start()
            
            self.start_button.config(state=tk.DISABLED)
            self.stop_button.config(state=tk.NORMAL)
            self.register_button.config(state=tk.NORMAL)
            self.clear_button.config(state=tk.NORMAL)
            self.status_text.set("Camera running. Align face and click 'Register'.")
            logging.info("Camera started.")
            
        except IOError as e:
            messagebox.showerror("Webcam Error", str(e))
            logging.error(f"Webcam failed to start: {e}")

    def stop_video_stream(self):
        if not self.is_running: return
        self.is_running = False
        if self.video_thread: self.video_thread.join(timeout=0.5) 
        if self.cap: self.cap.release()
        
        self.video_label.config(image=None)
        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        self.register_button.config(state=tk.DISABLED)
        self.clear_button.config(state=tk.DISABLED)
        self.status_text.set("Camera stopped.")
        
        self.clear_all_registrations()
        logging.info("Camera stopped.")

    def video_loop(self):
        while self.is_running:
            try:
                ret, frame = self.cap.read()
                if not ret:
                    time.sleep(0.1)
                    continue

                frame = cv2.flip(frame, 1)
                
                # Handle Calibration Logic inside the loop for accuracy
                if self.is_calibrating:
                    frame = self._handle_calibration_step_in_thread(frame)
                else:
                    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    frame, status_msg, list_msg = self.process_monitoring_logic(frame, rgb_frame)
                    
                    # Update UI
                    self.status_text.set(status_msg)
                    self.registered_list_var.set(list_msg)

                # Display Frame
                cv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                pil_img = Image.fromarray(cv_img)
                
                w = self.video_label.winfo_width()
                h = self.video_label.winfo_height()
                if w > 10 and h > 10:
                    pil_img = pil_img.resize((w, h), Image.Resampling.LANCZOS)
                
                imgtk = ImageTk.PhotoImage(image=pil_img)
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)

            except Exception as e:
                print(f"Loop Error: {e}")
                logging.error(f"Video loop error: {e}")
                self.is_running = False
            
            time.sleep(0.01)

    # --- Core Logic Methods ---

    def get_ear(self, eye_points, w, h):
        """Calculates Eye Aspect Ratio."""
        try:
            def dist(p1, p2):
                return np.linalg.norm(np.array([p1.x*w, p1.y*h]) - np.array([p2.x*w, p2.y*h]))
            v1 = dist(eye_points[1], eye_points[5])
            v2 = dist(eye_points[2], eye_points[4])
            h_dist = dist(eye_points[0], eye_points[3])
            if h_dist == 0: return 0.0
            return (v1 + v2) / (2.0 * h_dist)
        except:
            return 0.0

    def get_face_data(self, frame, results):
        """Parses MediaPipe results into usable data (box, EAR, center)."""
        faces = []
        h, w, _ = frame.shape
        if not results.multi_face_landmarks: return faces

        for landmarks in results.multi_face_landmarks:
            xs = [l.x for l in landmarks.landmark]
            ys = [l.y for l in landmarks.landmark]
            x_min, x_max = int(min(xs)*w), int(max(xs)*w)
            y_min, y_max = int(min(ys)*h), int(max(ys)*h)
            
            x_min, y_min = max(0, x_min-10), max(0, y_min-10)
            x_max, y_max = min(w, x_max+10), min(h, y_max+10)
            
            nose = landmarks.landmark[1]
            center = (int(nose.x*w), int(nose.y*h))

            LEFT_IDXS = [33, 159, 158, 133, 153, 145] 
            RIGHT_IDXS = [362, 386, 385, 263, 380, 374]

            left_pts = [landmarks.landmark[i] for i in LEFT_IDXS]
            right_pts = [landmarks.landmark[i] for i in RIGHT_IDXS]

            ear_left = self.get_ear(left_pts, w, h)
            ear_right = self.get_ear(right_pts, w, h)
            avg_ear = (ear_left + ear_right) / 2.0

            faces.append({
                'box': (x_min, y_min, x_max-x_min, y_max-y_min),
                'center': center,
                'ear': avg_ear
            })
        return faces

    # --- NEW: Thread-Safe Calibration Workflow ---

    def initiate_calibration(self):
        """Starts the burst calibration process."""
        if self.is_calibrating: return
        self.is_calibrating = True
        self.calibration_buffer = []
        self.status_text.set("CALIBRATING... KEEP EYES OPEN AND STILL!")
        # Disable buttons
        self.register_button.config(state=tk.DISABLED)
        logging.info("Calibration initiated.")

    def _handle_calibration_step_in_thread(self, frame):
        """
        Runs in Video Thread. Collects data.
        When done, schedules _finalize_calibration_in_main_thread on Main Thread.
        """
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(rgb_frame)
        faces = self.get_face_data(frame, results)
        
        h, w, _ = frame.shape
        
        if not faces:
            cv2.putText(frame, "NO FACE DETECTED", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            return frame

        # Find largest face
        target = sorted(faces, key=lambda f: f['box'][2]*f['box'][3], reverse=True)[0]
        
        # Draw Loading Bar
        progress = len(self.calibration_buffer) / CALIBRATION_FRAMES_REQUIRED
        bar_w = int(w * 0.6)
        cv2.rectangle(frame, (int(w*0.2), h-100), (int(w*0.2) + bar_w, h-70), (255, 255, 255), 2)
        cv2.rectangle(frame, (int(w*0.2), h-100), (int(w*0.2) + int(bar_w*progress), h-70), (0, 255, 0), -1)
        cv2.putText(frame, "CALIBRATING...", (int(w*0.2), h-110), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        self.calibration_buffer.append(target['ear'])

        if len(self.calibration_buffer) >= CALIBRATION_FRAMES_REQUIRED:
            # Calculation done in thread
            avg_ear = sum(self.calibration_buffer) / len(self.calibration_buffer)
            target_center = target['center']
            
            # CRITICAL: Stop calibrating logic in this thread
            self.is_calibrating = False
            
            # CRITICAL: Schedule GUI dialog on MAIN THREAD. Do not call simpledialog here.
            self.window.after(0, self._finalize_calibration_in_main_thread, target_center, avg_ear)

        return frame

    def _finalize_calibration_in_main_thread(self, face_center, avg_ear):
        """
        Runs on Main Thread. Opens dialog and updates variables.
        """
        # Logic Check
        if avg_ear < MIN_OPEN_EYE_EAR:
            messagebox.showwarning("Failed", f"Eyes detected as closed (EAR: {avg_ear:.2f}). Retry.")
            logging.warning(f"Calibration failed: EAR too low ({avg_ear:.2f})")
            self.register_button.config(state=tk.NORMAL)
            return

        # Calculate robust threshold
        threshold = min(avg_ear * THRESHOLD_CALIBRATION_FACTOR, MAX_POSSIBLE_THRESHOLD)
        
        # Dialog - Safe to call here
        name = simpledialog.askstring("Registration", f"Calibration Done (EAR: {avg_ear:.2f}).\nName:")
        
        if name and name not in self.registered_people:
            self.registered_people[name] = {
                'center': face_center,
                'threshold': threshold,
                'open_ear_baseline': avg_ear,
                'closed_frames': 0,
                'status': 'Active',
                'last_sound_time': 0
            }
            logging.info(f"Person registered: {name} with threshold {threshold:.3f}")
        else:
            logging.warning("Registration cancelled or duplicate name.")
            
        # Re-enable button
        self.register_button.config(state=tk.NORMAL)

    # --- Monitoring Logic ---

    def process_monitoring_logic(self, frame, rgb_frame):
        results = self.face_mesh.process(rgb_frame)
        faces = self.get_face_data(frame, results)
        
        if not self.registered_people:
            for f in faces:
                x, y, w, h = f['box']
                cv2.rectangle(frame, (x,y), (x+w, y+h), (100, 100, 100), 2)
            return frame, "Ready to Register.", "No one registered."

        active_faces = {i: f for i, f in enumerate(faces)}
        list_display = []

        for name, p_data in self.registered_people.items():
            best_dist = float('inf')
            best_idx = -1
            
            for idx, face in active_faces.items():
                dist = np.linalg.norm(np.array(face['center']) - np.array(p_data['center']))
                if dist < best_dist:
                    best_dist = dist
                    best_idx = idx
            
            if best_idx != -1 and best_dist < MAX_TRACKING_JUMP_PX:
                face = active_faces[best_idx]
                del active_faces[best_idx] 
                
                p_data['center'] = face['center']
                
                ear = face['ear']
                is_eyes_closed = ear < p_data['threshold']
                
                if is_eyes_closed:
                    p_data['closed_frames'] += 1
                else:
                    p_data['closed_frames'] = 0 
                
                if p_data['closed_frames'] > EYE_AR_CONSEC_FRAMES:
                    p_data['status'] = "!!! SLEEPING !!!"
                    self._trigger_alert(name, p_data)
                    color = (0, 0, 255)
                    cv2.putText(frame, "WAKE UP!", (face['box'][0], face['box'][1]-20), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)
                else:
                    p_data['status'] = "Active"
                    color = (0, 255, 0)

                x, y, w, h = face['box']
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.putText(frame, f"{name} (EAR: {ear:.2f})", (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
            else:
                p_data['status'] = "Lost"
                p_data['closed_frames'] = 0

            list_display.append(f"{name}: {p_data['status']}")

        status_msg = f"Monitoring {len(self.registered_people)} people."
        return frame, status_msg, "\n".join(list_display)

    def _trigger_alert(self, name, p_data):
        now = time.time()
        if now - p_data['last_sound_time'] > ALERT_SOUND_INTERVAL:
            logging.warning(f"Alert triggered for {name}!")
            threading.Thread(target=self._play_sound, daemon=True).start()
            p_data['last_sound_time'] = now

    def _play_sound(self):
        if os.path.exists(ALERT_SOUND_FILE_PATH):
            try:
                winsound.PlaySound(ALERT_SOUND_FILE_PATH, winsound.SND_FILENAME | winsound.SND_ASYNC)
            except:
                pass 

    def clear_all_registrations(self):
        self.registered_people = {}
        logging.info("All registrations cleared.")
        self.status_text.set("Registrations Cleared.")
        self.registered_list_var.set("No one registered.")

    def on_closing(self):
        if messagebox.askokcancel("Quit", "Exit Application?"):
            self.stop_video_stream()
            logging.info("Application closed.")
            self.window.destroy()

if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SleepingAlertApp(root)
        root.mainloop()
    except Exception as e:
        logging.critical(f"Critical crash: {e}")
        print(f"Crash: {e}")

Loop Error: dictionary changed size during iteration
